# RAG SDK
Creating a RAG sdk for faster iteration and exploring the RAG competitions like Wattbot

## Defining imports

In [ ]:
import os
import openai
import pandas as pd
import fastcore.all as fc
import contextkit.read as rd
import fireworks as fw
import braintrust
import time

import ast
from types import SimpleNamespace
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams, PointStruct
from fastembed import SparseTextEmbedding

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
em = "nomic-ai/nomic-embed-text-v1.5" # embedding model
# em = "accounts/fireworks/models/qwen3-embedding-8b"

## Loading data files (related to competition)

In [ ]:
data_path = fc.Path('wattbot_data')
metadata = pd.read_csv(data_path/'metadata.csv', encoding='latin-1', dtype={'year': object})
qa = pd.read_csv(data_path/'train_QA.csv')
tst = pd.read_csv(data_path/'test_Q.csv')

In [ ]:
metadata.head()

,id,type,title,year,citation,url
0,amazon2023,report,2023 Amazon Sustainability Report,2023,Amazon Staff. (2023). Amazon Sustainability Re...,https://sustainability.aboutamazon.com/2023-am...
1,chen2024,paper,Efficient Heterogeneous Large Language Model D...,2024,"Shaoyuan Chen, Wencong Xiao, Yutong Lin, Mingx...",https://arxiv.org/pdf/2405.01814
2,chung2025,paper,The ML.ENERGY Benchmark: Toward Automated Infe...,2025,"Jae-Won Chung, Jiachen Liu, Jeff J. Ma, Ruofan...",https://arxiv.org/pdf/2505.06371
3,cottier2024,paper,The Rising Costs of Training Frontier AI Models,2024,"Ben Cottier, Robi Rahman, Loredana Fattorini, ...",https://arxiv.org/pdf/2405.21015
4,dodge2022,paper,Measuring the Carbon Intensity of AI in Cloud ...,2022,"Jesse Dodge, Taylor Prewitt, Remi Tachet Des C...",https://arxiv.org/pdf/2206.05229


In [ ]:
metadata.dtypes


id          object
type        object
title       object
year        object
citation    object
url         object
dtype: object

How to assign dtype object for column year when reading csv file?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can specify the data type when reading the CSV file using the `dtype` parameter in `pd.read_csv()`. Here's how:

```python
metadata = pd.read_csv(data_path/'metadata.csv', encoding='latin-1', dtype={'year': object})
```

This tells pandas to read the 'year' column as an object (string) type instead of trying to infer it as an integer.

Would you like to reload your metadata with this change, or do you have a specific reason for wanting 'year' as an object type?

## Document Processor

In [ ]:
class Chunk(fc.NS):
    def __init__(self, chunk_id, doc_id, text, **metadata):
        super().__init__(chunk_id=chunk_id, doc_id=doc_id, text=text, **metadata)
    
    def __repr__(self):
        meta = "\n".join([f"\033[1;35m{k}\033[0m: {str(v)[:100]}" for k, v in self.__dict__.items()])
        return f"{meta}"



In [ ]:
Chunk('WattBot', 0, 0, **{'title': 'RAG'})

chunk_id: WattBot
doc_id: 0
text: 0
title: RAG

How to make the keys bold?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To make the keys bold in the `__repr__` output, you can wrap them in ANSI escape codes or markdown-style formatting. Here's a simple approach using ANSI codes for terminal output:

```python
def __repr__(self):
    meta = "\n".join([f"\033[1m{k}\033[0m: {v}" for k, v in self.__dict__.items() if k != '__stored_args__'])
    return f"{meta}"
```

The `\033[1m` starts bold text and `\033[0m` resets it.

Would you like to try this, or would you prefer a different formatting approach?

Can it be colorize?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! You can use different ANSI color codes. For example:

```python
f"\033[1;34m{k}\033[0m: {v}"  # Bold blue
f"\033[1;32m{k}\033[0m: {v}"  # Bold green
f"\033[1;35m{k}\033[0m: {v}"  # Bold magenta
```

The format is `\033[1;COLORm` where common color codes are:
- 31: red
- 32: green
- 33: yellow
- 34: blue
- 35: magenta
- 36: cyan

Which color would you like to try?

In [ ]:
import fastcore.all as fc
import contextkit.read as rd

class DocumentProcessor:
    def __init__(self, data_path='wattbot_data', chunk_size=1500, step=1400):
        fc.store_attr()
    
    def stream(self, doc_id, metadata=None):
        content = rd.read_pdf(fc.Path(self.data_path)/f'{doc_id}.pdf')
        for i in range(0, len(content), self.step): yield Chunk(text=content[i:i+self.chunk_size], chunk_id=i, doc_id=doc_id, **(metadata or {}))

    def process(self, doc_id, metadata=None):
        return fc.L(self.stream(doc_id, metadata=metadata))

In [ ]:
doc_id, doc_type, doc_url, doc_title, doc_cite, doc_year = fc.getattrs(metadata.iloc[0], 'id', 'type', 'url', 'title', 'citation', 'year')
doc_proc = DocumentProcessor()
extras = {'type': doc_type, 'url': doc_url, 'title': doc_title, 'citation': doc_cite, 'year': doc_year}
chunks = fc.L(doc_proc.process(doc_id, metadata=extras))
chunks[0]

chunk_id: 0
doc_id: amazon2023
text: Amazon 
Sustainability 
Report
2023 Contents
Overview
3 Introduction
4 A Letter from Our Chief 
Sust
type: report
url: https://sustainability.aboutamazon.com/2023-amazon-sustainability-report.pdf
title: 2023 Amazon Sustainability Report
citation: Amazon Staff. (2023). Amazon Sustainability Report. https://sustainability.aboutamazon.com/2023-amaz
year: 2023

In [ ]:
def process_docs(data_path='wattbot_data'):
    metadata = pd.read_csv(fc.Path(data_path)/'metadata.csv', encoding='latin-1')
    doc_proc = DocumentProcessor()
    all_chunks = metadata.apply(lambda r: doc_proc.process(r['id'], metadata=fc.attrdict(r, 'type', 'url', 'title', 'citation', 'year')), axis=1).tolist()
    return fc.L(all_chunks).concat()

In [ ]:
all_chunks = process_docs()
all_chunks[0]

chunk_id: 0
doc_id: amazon2023
text: Amazon 
Sustainability 
Report
2023 Contents
Overview
3 Introduction
4 A Letter from Our Chief 
Sust
type: report
url: https://sustainability.aboutamazon.com/2023-amazon-sustainability-report.pdf
title: 2023 Amazon Sustainability Report
citation: Amazon Staff. (2023). Amazon Sustainability Report. https://sustainability.aboutamazon.com/2023-amaz
year: 2023

How to save all the chunks so that I dont generate again?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You have several options for saving the chunks:

1. **Pickle** - Fast and preserves Python objects:
```python
fc.save_pickle('all_chunks.pkl', all_chunks)
```

2. **JSON** - More portable and readable:
```python
import json
with open('all_chunks.json', 'w') as f:
    json.dump([chunk.__dict__ for chunk in all_chunks], f)
```

3. **Parquet** - Efficient for large datasets:
```python
pd.DataFrame([chunk.__dict__ for chunk in all_chunks]).to_parquet('all_chunks.parquet')
```

Which format would you prefer, or would you like to know more about the tradeoffs?

How to load the chunks?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

For each saving method, here's how to load:

1. **Pickle**:
```python
all_chunks = fc.load_pickle('all_chunks.pkl')
```

2. **JSON**:
```python
with open('all_chunks.json', 'r') as f:
    all_chunks = fc.L([Chunk(**d) for d in json.load(f)])
```

3. **Parquet**:
```python
all_chunks = fc.L([Chunk(**row) for row in pd.read_parquet('all_chunks.parquet').to_dict('records')])
```

Which format did you use to save them?

## Embedding

In [ ]:
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

Fetching 18 files:   6%|▌         | 1/18 [00:00<00:07,  2.34it/s]

Fetching 18 files:  11%|█         | 2/18 [00:00<00:03,  4.22it/s]

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 39.63it/s]

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 26.24it/s]

In [ ]:
class Embedder:
    def __init__(self, dclient, dmodel, smodel=None, bs=50):
        fc.store_attr()
    
    def create_dembs(self, texts):
        data = fc.mapped(lambda o: self.dclient.embeddings.create(model=self.dmodel, input=o), fc.chunked(texts, chunk_sz=self.bs)).attrgot('data')
        return data.concat().attrgot('embedding')

    def create_sembs(self, texts):
        if not self.smodel: raise ValueError("Sparse model not initialized")
        return list(self.smodel.embed(texts))
        

In [ ]:
openai_cli = openai.OpenAI(base_url = "https://api.fireworks.ai/inference/v1", api_key=os.environ['FIREWORKS_API_KEY'])
embedder = Embedder(openai_cli, em, smodel=sparse_model)
embds = embedder.create_dembs(chunks.attrgot('text'))
len(embds), len(embds[0])

(322, 768)

I have an api key for gemini and want to use for embedding model with the openai spec
https://ai.google.dev/gemini-api/docs/openai

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To use Gemini's embedding model with the OpenAI spec, you'll need to:

1. Change the base URL to Gemini's endpoint
2. Use your Gemini API key
3. Specify a Gemini embedding model name

Here's what to modify:

```python
openai_cli = openai.OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.environ['GEMINI_API_KEY']
)
```

For the embedding model, Gemini supports models like `text-embedding-004`. You'd set:

```python
em = "text-embedding-004"
```

Have you already set your `GEMINI_API_KEY` environment variable, or would you like help with that?

I am getting info logs when calling to gemini 
2025-11-29 15:22:21,210 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/embeddings "HTTP/1.1 200 OK"

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can suppress those INFO logs by adjusting the logging level for the HTTP client. Add this at the top of your notebook:

```python
import logging
logging.getLogger("httpx").setLevel(logging.WARNING)
```

This will only show warnings and errors, not info messages. Would you like to try this?

In [ ]:
all_dense_embds = embedder.create_dembs(all_chunks.attrgot('text'))
len(all_dense_embds), len(all_dense_embds[0])

(1899, 768)

In [ ]:
all_sparse_embds = embedder.create_sembs(all_chunks.attrgot('text'))
len(all_sparse_embds), len(all_sparse_embds[0].values)

(1899, 137)

## Qdrant

In [ ]:
QHOST = "https://76eb75ff-5ca3-4a68-93d4-ae19a9c169a4.us-west-2-0.aws.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

In [ ]:
class QdrantStore:
    def __init__(self, url, key, collection_name, use_hybrid=False):
        fc.store_attr()
        self.cli = QdrantClient(url=url, api_key=key)
        self.points = []

    def create_or_replace_collection(self, vs, replace=False, distance=Distance.COSINE):
        if replace: self.cli.delete_collection(collection_name=self.collection_name)
        sparse_vectors_config = None
        if not self.cli.collection_exists(self.collection_name): 
            if self.use_hybrid: sparse_vectors_config={"sparse": models.SparseVectorParams()}
            self.cli.create_collection(collection_name=self.collection_name, 
                vectors_config={"dense": VectorParams(size=vs, distance=distance)}, 
                sparse_vectors_config=sparse_vectors_config)
        return self
    

    def _sv(self, semb):
        return models.SparseVector(values=semb.values.tolist(), indices=semb.indices.tolist())

    def create_hybrid_points(self, chunks, dembs, sembs, start_id=0):
        return fc.L(chunks, dembs, sembs).zip().enumerate().map(lambda ele: PointStruct(id=start_id+ele[0], vector={"dense": ele[1][1], "sparse": self._sv(ele[1][2])}, payload=ele[1][0].__dict__))

    def create_points(self, chunks, dembs, sembs=None, start_id=0):
        if self.use_hybrid and sembs: 
            self.points = self.create_hybrid_points(chunks, dembs, sembs, start_id)
        else: 
            self.points = fc.L(chunks, dembs).zip().enumerate().map(lambda ele: PointStruct(id=start_id+ele[0], vector={"dense": ele[1][1]}, payload=ele[1][0].__dict__))
        return self
    
    def upsert(self, bs=100):
        fc.mapped(lambda o: self.cli.upsert(collection_name=self.collection_name, points=list(o)), fc.chunked(self.points, chunk_sz=bs))
        return self

    def hybrid_search(self, query_demb, query_semb, limit=5):
        return self.cli.query_points(collection_name=self.collection_name,
            prefetch=[
                models.Prefetch(query=query_demb, using="dense", limit=limit*2),
                models.Prefetch(query=self._sv(query_semb), using="sparse", limit=limit*2)
            ],
            query=models.FusionQuery(fusion=models.Fusion.RRF),
            limit=limit
        )

    def search(self, query_demb, query_semb=None, limit=5):
        if self.use_hybrid and query_semb:
            results = self.hybrid_search(query_demb, query_semb, limit)
        else:
            results = self.cli.query_points(collection_name=self.collection_name, query=query_vector, limit=limit)
        return results.points

Do we really require qdrant datastore for storing 1899 embeddings and searching? Will a brute force search work which does search across all the embedding

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! For 1,899 embeddings, brute force search would absolutely work and might even be simpler and faster than using Qdrant.

With modern hardware, computing cosine similarity across ~2,000 vectors is very quick (milliseconds). Vector databases like Qdrant are mainly beneficial when you have:
- Hundreds of thousands or millions of vectors
- Need for persistence across sessions
- Advanced filtering on metadata
- Multiple concurrent users

For your use case, you could use simple numpy/scipy operations:

```python
from sklearn.metrics.pairwise import cosine_similarity
```

Would you like to try implementing a simple brute force search to compare? Or are you planning to scale up significantly and want to keep Qdrant for future growth?

And what about the keyword search?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good point! For keyword search (like BM25), you have a few options without Qdrant:

1. **Simple approach**: Use libraries like `rank-bm25` which does in-memory BM25 scoring
2. **Hybrid with numpy**: Store sparse embeddings as scipy sparse matrices and compute similarities
3. **Full-text search**: Use SQLite's FTS5 for keyword search

For 1,899 documents, all of these would be fast enough. The sparse embeddings you're generating with `Qdrant/bm25` model could be stored as scipy sparse matrices and searched with simple dot products.

Are you interested in implementing a lightweight alternative, or do you prefer keeping Qdrant for the convenience of having both dense and sparse search in one place?

In [ ]:
cname = 'wattbot_docs_nomic_768_sparse'
# cname = 'wattbot_docs_qwen_8b'
qs = QdrantStore(QHOST, api_key, cname, use_hybrid=True).create_or_replace_collection(768, replace=True)
qs.create_points(all_chunks, all_dense_embds, all_sparse_embds).upsert()

<__main__.QdrantStore>

In [ ]:
qs.cli.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='wattbot_docs_nomic_768'), CollectionDescription(name='wattbot_documents_768'), CollectionDescription(name='wattbot_docs_qwen_8b'), CollectionDescription(name='colbert_chunks'), CollectionDescription(name='wattbot_documents'), CollectionDescription(name='wattbot_docs_nomic_768_sparse')])

Instead of the qdrant database I save the dense and sparse embeddings locally and do similarity search then combine the results

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's a solid approach for this dataset size! You'd essentially be building a lightweight hybrid search system.

The basic steps would be:
1. Save dense embeddings (numpy array)
2. Save sparse embeddings (scipy sparse matrix or list)
3. For search: compute similarities separately for dense and sparse
4. Combine results using a fusion method (like RRF - Reciprocal Rank Fusion)

Would you like me to help you implement this step by step, or would you prefer to try building it yourself with guidance on specific parts as needed?

Yes
I would need to help with saving the sparse embedding and compute similarity for them

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start with saving the sparse embeddings.

Since your sparse embeddings have `.values` and `.indices` attributes (from the fastembed model), you have a couple of options:

1. **Scipy sparse matrix** - efficient for similarity computation
2. **List of dicts** - simpler, keeps the structure as-is

For scipy, you'd convert to CSR format which is efficient for similarity searches. For the list approach, you'd just pickle the embeddings directly.

Which approach sounds better to you, or would you like to try the scipy sparse matrix route since it's more efficient for similarity computation?

What is CSR format

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

CSR stands for **Compressed Sparse Row** format. It's a way to efficiently store sparse matrices (matrices with mostly zeros) by only storing the non-zero values.

Instead of storing a full matrix like:
```
[0, 0, 3, 0, 5]
[2, 0, 0, 0, 0]
[0, 4, 0, 1, 0]
```

CSR stores three arrays:
- **values**: the non-zero values `[3, 5, 2, 4, 1]`
- **indices**: which column each value is in
- **indptr**: where each row starts

This saves memory and makes operations like dot products much faster when you have lots of zeros (which is typical for sparse embeddings like BM25).

Does this make sense? Would you like to proceed with using scipy's CSR format for your sparse embeddings?

Okay lets do that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Let's start with converting your sparse embeddings to CSR format.

First, you'll need scipy's sparse module:
```python
from scipy.sparse import csr_matrix
```

To convert your `all_sparse_embds` (which are fastembed sparse vectors) to a CSR matrix, you'll need to extract the indices and values from each embedding and stack them into a matrix.

Try creating a function that takes one sparse embedding and returns the indices and values. What do you get when you look at `all_sparse_embds[0].indices` and `all_sparse_embds[0].values`?

I created the function def get_val_idx(s): return s.indices, s.values

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good start! Now you have a way to extract the indices and values from each sparse embedding.

The next step is to build the CSR matrix from all your embeddings. You'll need to:
1. Determine the maximum index across all embeddings (this will be the number of columns)
2. Stack all the indices and values together
3. Create the CSR matrix

Try finding the maximum index first. What do you get when you check the max index across all your sparse embeddings?

THe highest value of indices are 30151

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! So your CSR matrix will have 30,152 columns (since indices are 0-based).

Now you need to build the CSR matrix. Scipy's `csr_matrix` can be constructed using:
```python
csr_matrix((data, indices, indptr), shape=(n_rows, n_cols))
```

Where:
- `data` = all values concatenated
- `indices` = all column indices concatenated  
- `indptr` = array marking where each row starts in the data array

Try creating the `data` and `indices` arrays first by concatenating all the values and indices from your sparse embeddings. What approach will you use to concatenate them?

#| export
What is the difference between sparse embedding and lexical search ?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! They're related but different concepts:

**Lexical search** is the *method* - it matches documents based on exact words/tokens (like BM25, TF-IDF). It looks for keyword overlap between query and documents.

**Sparse embeddings** are a *representation* - they encode text as vectors with mostly zeros, where non-zero positions often correspond to specific tokens or n-grams. They can be used for lexical-style search but in vector form.

So the sparse embeddings you're creating with the `Qdrant/bm25` model are essentially a vectorized way to do BM25 lexical search - they capture keyword matching but in a format that works with vector similarity operations.

Does that distinction make sense?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Now, back to building your CSR matrix - do you want to continue with concatenating the data and indices arrays, or would you like to explore something else about your RAG system first?

lets do a simple lexical search instead of the model

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good idea! For simple lexical search, you have a few options:

1. **BM25 library** (`rank-bm25`) - classic keyword scoring
2. **TF-IDF with sklearn** - term frequency approach
3. **SQLite FTS5** - built-in full-text search

BM25 is generally considered best for search/retrieval tasks. Would you like to try the `rank-bm25` library, or explore one of the other options?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! First, you'll need to install the library:

```python
!pip install rank-bm25
```

Then import it:
```python
from rank_bm25 import BM25Okapi
```

BM25 needs tokenized documents to build its index. You'll need to split your chunk texts into tokens (words). How would you like to tokenize - simple `split()` or something more sophisticated?

When adding the package rank-bm25
I am getting the following error

 No solution found when resolving dependencies for split (python_full_version == '3.9.*'):
  ╰─▶ Because the requested Python version (>=3.9) does not satisfy Python>=3.10 and contextkit==0.0.10 depends on Python>=3.10, we can conclude that contextkit==0.0.10 cannot be used.
      And because only contextkit<=0.0.10 is available and your project depends on contextkit>=0.0.10, we can conclude that your project's requirements are unsatisfiable.

      hint: The `requires-python` value (>=3.9) includes Python versions that are not supported by your dependencies (e.g., contextkit==0.0.10 only supports >=3.10). Consider using a more restrictive
      `requires-python` value (like >=3.10).
  help: If you want to add the package regardless of the failed resolution, provide the `--frozen` flag to skip locking and syncing.
(wattbot) (base) MacBook-Pro:wattbot anubhavmaity$ 


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

This error is about your Python version and project dependencies, not specifically about `rank-bm25`. 

Your project requires `contextkit>=0.0.10` which needs Python 3.10+, but your project's `requires-python` is set to `>=3.9`.

You have two options:

1. **Update your project's Python requirement** to `>=3.10` in your `pyproject.toml` or similar config file
2. **Use the `--frozen` flag** as suggested: `uv pip install rank-bm25 --frozen` (though this skips dependency resolution)

Which Python version are you actually running? You can check with `python --version`.

okay I was able to install rank-bm25, now what is next

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Now you need to tokenize your chunk texts to build the BM25 index.

Try creating a list of tokenized documents from your chunks. For each chunk's text, split it into words. What method will you use - simple `split()` or something else?

simple split will do

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Try creating the tokenized corpus:

```python
tokenized_corpus = [chunk.text.lower().split() for chunk in all_chunks]
```

The `.lower()` helps with case-insensitive matching. Once you have this, you can create the BM25 index. Give it a try and let me know what you get!

got the tokenized corpus what is next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Excellent! Now create the BM25 index:

```python
bm25 = BM25Okapi(tokenized_corpus)
```

This builds the index from your tokenized documents. Once you have this, you'll be able to search by tokenizing a query and using `bm25.get_scores()` or `bm25.get_top_n()`.

Have you created the index?

What should I pass to get_top_n

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

`get_top_n()` takes three main arguments:

1. **tokenized_query** - your search query split into tokens (like you did for the corpus)
2. **documents** - the original documents/chunks you want returned
3. **n** - how many top results you want

So it would look something like:
```python
bm25.get_top_n(tokenized_query, all_chunks, n=5)
```

What query would you like to test with?

How should I create a new class to have bm25 and do the cosine similarity across all vectos?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good thinking to organize this into a class! Here's a structure to consider:

```python
class HybridSearch:
    def __init__(self, chunks, dense_embds, tokenized_corpus):
        # Store your data
        # Initialize BM25
        # Store dense embeddings as numpy array
```

You'll need methods like:
- `lexical_search(query, top_k)` - for BM25
- `dense_search(query_emb, top_k)` - for cosine similarity
- `hybrid_search(query, query_emb, top_k)` - combines both

For cosine similarity, you can use `sklearn.metrics.pairwise.cosine_similarity` or just normalize vectors and use dot product.

Would you like to start building this class yourself, or would you like me to write it out for you?

Can you please show me how to do the cosine similarity?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! Here are two approaches for cosine similarity:

**Option 1 - Using sklearn:**
```python
from sklearn.metrics.pairwise import cosine_similarity
scores = cosine_similarity([query_emb], dense_embds)[0]
```

**Option 2 - Using numpy (faster for normalized vectors):**
```python
import numpy as np
# Normalize vectors first
dense_embds_norm = dense_embds / np.linalg.norm(dense_embds, axis=1, keepdims=True)
query_norm = query_emb / np.linalg.norm(query_emb)
scores = dense_embds_norm @ query_norm
```

Both give you similarity scores for each document. To get top-k results, you'd use `np.argsort(scores)[-k:][::-1]` to get the indices.

Which approach would you like to use in your class?

## Retriever

In [ ]:
class Retriever:
    def __init__(self, embedder, vector_store):
        fc.store_attr()
    
    def retrieve(self, query, limit=5):
        query_demb = self.embedder.create_dembs(texts=[query])[0]
        query_semb = self.embedder.create_sembs(texts=[query])[0] if self.embedder.smodel else None
        return self.vector_store.search(query_demb, query_semb, limit=limit)

In [ ]:
query_text = "What are Amazon's carbon emissions goals?"
retriever = Retriever(embedder, qs)
retriever.retrieve(query_text, limit=1)[0].payload['text'][:200]

'to build Climate Pledge Arena, the first International \nLiving Future Institute zero-carbon certified sports and \nentertainment venue in the world.\nCarbon \nThe science is clear. Significant carbon emi'

To support creating the embeddings for both sparse and dense. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You need to update the `Retriever` class to handle both dense and sparse embeddings. The error shows it's calling `create_embeddings` which doesn't exist - your `Embedder` has `create_dembs` and `create_sembs` instead.

Here's how you can modify the `retrieve` method to work with your hybrid setup:

```python
def retrieve(self, query, limit=5):
    query_demb = self.embedder.create_dembs(texts=[query])[0]
    query_semb = self.embedder.create_sembs(texts=[query])[0] if self.embedder.smodel else None
    # Now you need to decide how to combine them for search
```

However, I notice your `QdrantStore.search` method currently only takes a single `query_vector`. For hybrid search with both dense and sparse vectors, you'll need to update that method too.

Would you like help modifying the search method to support hybrid queries, or do you want to work on that yourself?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! For hybrid search in Qdrant, you need to use the `query_points` method with a `prefetch` parameter to combine dense and sparse searches.

Here's the key structure:

```python
def search(self, query_demb, query_semb=None, limit=5):
    if self.use_hybrid and query_semb:
        # Use prefetch for hybrid search
        results = self.cli.query_points(
            collection_name=self.collection_name,
            prefetch=[
                models.Prefetch(query=query_demb, using="dense", limit=limit*2),
                models.Prefetch(query=self._sv(query_semb), using="sparse", limit=limit*2)
            ],
            query=models.FusionQuery(fusion=models.Fusion.RRF),
            limit=limit
        )
```

The `prefetch` retrieves candidates from both dense and sparse vectors, then `FusionQuery` with RRF (Reciprocal Rank Fusion) combines the results.

Would you like to complete the rest of the method to handle the non-hybrid case?

In [ ]:
query_text = "What dataset of 5,842 labeled entries was used to test energy-efficient large language models in the financial domain?"
retriever.retrieve(query_text, limit=3)[2]

ScoredPoint(id=1864, version=18, score=0.33333334, payload={'chunk_id': 8400, 'doc_id': 'zschache2025', 'text': ' on a single\nbenchmark, we extend these efforts to a greater number of partially very large models\ndeployed to a practical inference scenario.\nThis makes our study very similar to the one by Alizadeh et al. (2025), who inves-\ntigated the trade-offs between accuracy and energy consumption when deploying large\nlanguage models (LLMs) for software development tasks. Besides the finding that\nlarger LLMs with higher energy consumption do not always yield significantly bet-\nter accuracy, the authors demonstrated that architectural factors, such as feedforward\nlayer size and transformer block count, directly correlate with energy usage.\n3 Finally, Luccioni et al. (2024) provide one of the most comprehensive analyses\nof energy consumption during ML model inference. Their study systematically com-\npared the energy costs of 88 models across 10 tasks and 30 datasets, includin

How to remove the __stored_args__ from the ScoredPoint above? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The `__stored_args__` is being added to the payload because it's part of the `Chunk` object's `__dict__`. You can filter it out when creating the points.

In your `QdrantStore.create_points` method, modify the payload creation to exclude it:

```python
payload={k: v for k, v in ele[1][0].__dict__.items() if k != '__stored_args__'}
```

This filters out `__stored_args__` before storing the payload. Would you like to update both the `create_points` and `create_hybrid_points` methods with this change?

Cant we modify the Chunk class to not store the __stored_args__?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes, absolutely! That's a cleaner solution. The `__stored_args__` comes from `fastcore`'s `store_attr()` function. You can either:

1. Override `__dict__` to exclude it
2. Delete it after calling `store_attr()`
3. Use a property/method to return a clean dict

The simplest approach would be to add this line in your `Chunk.__init__`:

```python
del self.__stored_args__
```

Would you like to try that, or would you prefer a different approach?

I want to search for the text "5,842 labeled entries" in all_chunks

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can search for that text using a list comprehension or the `filter` method. Here's a simple approach:

```python
matching_chunks = [chunk for chunk in all_chunks if '5,842 labeled entries' in chunk.text]
```

This will give you all chunks that contain that exact phrase. Would you like to try this and see what you find?

In [ ]:
khan2025_chunks = DocumentProcessor().process('khan2025')
[chunk for chunk in khan2025_chunks if '5,842 entries' in chunk.text]

[chunk_id: 18200
 doc_id: khan2025
 text: sed language model
 tuned for general-purpose tasks.
 • Mistral-7B-Instruct-v0.3: An instruction-tuned]

It does not return the result with 5,482 labeled entries. How to make sure it returns that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The issue is that you're searching for the exact phrase '5,842 entries' but the actual text might have different formatting - perhaps it says '5,842 labeled entries' or has extra spaces or punctuation.

Try searching with just the number:

```python
[chunk for chunk in khan2025_chunks if '5,842' in chunk.text]
```

This will find any chunk containing that number, regardless of the surrounding words. Does that return the result you're looking for?

In [ ]:
[chunk for chunk in khan2025_chunks if '5,842' in chunk.text]

[chunk_id: 18200
 doc_id: khan2025
 text: sed language model
 tuned for general-purpose tasks.
 • Mistral-7B-Instruct-v0.3: An instruction-tuned]

In [ ]:
# [chunk for chunk in all_chunks if 'JetMoE-8B' in chunk.text]

Why the retrieve function is returning the result as 2nd index instead of first?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good question! There are a few possible reasons why the most relevant chunk isn't ranking first:

1. **Embedding similarity** - The query embedding might be more similar to other chunks based on semantic meaning, not just keyword matching
2. **Hybrid search weighting** - The RRF (Reciprocal Rank Fusion) is combining dense and sparse results, and the sparse (BM25) scoring might be favoring other chunks
3. **Context around the keywords** - Other chunks might have more context that semantically matches your query

To debug this, you could:
- Check the scores of the top 3 results to see how close they are
- Try adjusting the fusion weights or prefetch limits in your hybrid search
- Compare dense-only vs sparse-only vs hybrid results

Would you like to investigate any of these possibilities?

## Generator

In [ ]:
class Generator:
    def __init__(self, llm_client, prompt_template):
        fc.store_attr()
    
    def create_context(self, retrieved_chunks):
        return "\n\n".join([f"Doc ID: {chunk.payload['doc_id']}\nURL: {chunk.payload['url']}\nTitle: {chunk.payload['title']}\nText: {chunk.payload['text']}" for chunk in retrieved_chunks])
    
    def format_prompt(self, query, context):
        return self.prompt_template.build(context=context, question=query)
    
    def generate(self, query, retrieved_chunks):
        context = self.create_context(retrieved_chunks)
        formatted_prompt = self.format_prompt(query, context)
        response = self.llm_client.chat.completions.create(formatted_prompt['messages'], response_format={"type": "json_object"})
        time.sleep(1)
        return fc.loads(response.choices[0].message.content)

In [ ]:
# llm = fw.LLM(model="accounts/fireworks/models/deepseek-v3p1",  deployment_type="auto")
# llm = fw.LLM(model="accounts/fireworks/models/deepseek-r1-0528",  deployment_type="auto")
# model = 'accounts/fireworks/models/kimi-k2-instruct-0905'
# model = 'accounts/fireworks/models/llama-v3p1-70b-instruct'
model = 'accounts/fireworks/models/deepseek-v3p1'

# model = 'accounts/fireworks/models/kimi-k2-thinking'

llm = fw.LLM(model=model,  deployment_type="auto")

I want to use a gemini model with openai spec

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You can use Gemini models with the OpenAI spec by creating an OpenAI client with Gemini's base URL, similar to what you did for embeddings earlier.

For the `Generator` class, you'd pass in an OpenAI client configured for Gemini:

```python
gemini_client = openai.OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.environ['GEMINI_API_KEY']
)
```

Then for the model name, you'd use something like `"gemini-1.5-pro"` or `"gemini-1.5-flash"`.

Have you already set up your Gemini API key, or would you like help with that part?

How to pass the model name?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

When you create the chat completion in your `Generator.generate` method, you pass the model name to the `create` call:

```python
response = self.llm_client.chat.completions.create(
    model="gemini-1.5-pro",  # or "gemini-1.5-flash"
    messages=formatted_prompt['messages'],
    response_format={"type": "json_object"}
)
```

Note: Check if Gemini supports the `response_format` parameter - not all models/providers support structured JSON output this way.

Would you like to test this with your Gemini client?

How to enable the thinking model for kimi-k2-thinking model? And is there a prefill the model with {

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

For thinking models like `kimi-k2-thinking`, you typically need to:

1. **Enable thinking mode** - Check if there's a parameter like `enable_thinking=True` or similar in the Fireworks API
2. **Prefill with `{`** - Yes, some models support prefilling the assistant's response to force JSON output

For prefilling, you'd add an assistant message with just `{` to force JSON formatting:

```python
messages = [
    {"role": "user", "content": "your prompt"},
    {"role": "assistant", "content": "{", "prefix": True}
]
```

The exact parameter name might vary. Have you checked the Fireworks documentation for `kimi-k2-thinking` specific parameters? Would you like me to help you look that up?

Yes please

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->



<details class='tool-usage-details'>

```json
{
  "id": "toolu_01QueTq9m7VMedbMSWThKkaY",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://docs.fireworks.ai/guides/querying-text-models"
    }
  },
  "result": "'Skip to main content\\n\\n[Fireworks AI Docs home page![light logo](https://mintcdn.com/fireworksai/sTHhFfY93wc80BaS/logo/logo.svg?fit=max&auto=format&n=sTHhFfY93wc80BaS&q=85&s=33df07dcf4adc8ba74a3d9c6e5153b3a)![dark logo](https://mintcdn.com/fireworksai/sTHhFfY93wc80BaS/logo/logo.svg?fit=max&auto=format&n=sTHhFfY93wc80BaS&q=85&s=33df07dcf4adc8ba74a3d9c6e5153b3a)](/)\\n\\n[Documentation](/getting-started/introduction)[API & SDK Reference](/api-reference/introduction)[CLI Reference](/tools-sdks/firectl/firectl)[Demos](https://demos.fireworks.ai)[Resources](/getting-started/concepts)\\n\\n  * [Community](https://discord.gg/fireworks-ai)\\n  * [Status](https://status.fireworks.ai/)\\n  * [Dashboard](https://app.fireworks.ai/login)\\n  * [Dashboard](https://app.fireworks.ai/login)\\n\\nSearch...\\n\\nNavigation\\n\\nModels & Inference\\n\\nText Models\\n\\nSearch...\\n\\n\u2318K\\n\\n##### Get Started\\n\\n  * [Overview](/getting-started/introduction)\\n  * [Serverless Quickstart](/getting-started/quickstart)\\n  * [Deployments Quickstart](/getting-started/ondemand-quickstart)\\n  * [Which model should I use?](/guides/recommended-models)\\n\\n##### Deployments\\n\\n  * [Deployments Overview](/guides/ondemand-deployments)\\n  * [Autoscaling](/deployments/autoscaling)\\n  * [Custom Models](/models/uploading-custom-models)\\n  * [Quantization](/models/quantization)\\n  * Advanced\\n\\n##### Models & Inference\\n\\n  * [Text Models](/guides/querying-text-models)\\n  * [Vision Models](/guides/querying-vision-language-models)\\n  * [Speech to Text Models](/guides/querying-asr-models)\\n  * [Embeddings & Reranking](/guides/querying-embeddings-models)\\n  * [Tool Calling](/guides/function-calling)\\n  * [Batch API](/guides/batch-inference)\\n  * Advanced\\n\\n##### Fine Tuning\\n\\n  * [Fine Tuning Overview](/fine-tuning/finetuning-intro)\\n  * Supervised Fine Tuning\\n\\n  * Reinforcement Fine Tuning\\n\\n  * [Deploying Fine Tuned Models](/fine-tuning/deploying-loras)\\n\\n##### Administration\\n\\n  * [Managing users](/accounts/users)\\n  * [Custom SSO](/accounts/sso)\\n  * [Service Accounts](/accounts/service-accounts)\\n  * [Rate Limits & Quotas](/guides/quotas_usage/rate-limits)\\n  * [Exporting Billing Metrics](/accounts/exporting-billing-metrics)\\n\\n##### Security & Compliance\\n\\n  * [Data Security & Privacy](/guides/security_compliance/data_security)\\n  * [Zero Data Retention](/guides/security_compliance/data_handling)\\n  * [Audit & Access Logs](/guides/security_compliance/audit_logs)\\n\\n##### Integrations\\n\\n  * [Cloud Integrations](/ecosystem/integrations)\\n  * [Agent Frameworks](/ecosystem/integrations/agent-frameworks)\\n  * [MLOps & Observability](/ecosystem/integrations/mlops-observability)\\n\\nOn this page\\n\\n  * Chat Completions API\\n  * Alternative query methods\\n  * Querying dedicated deployments\\n  * Common patterns\\n  * Multi-turn conversations\\n  * System prompts\\n  * Streaming responses\\n  * Async requests\\n  * Usage & performance tracking\\n  * Advanced capabilities\\n  * Configuration & debugging\\n  * Understanding tokens\\n  * OpenAI SDK Migration\\n  * Next steps\\n\\nModels & Inference\\n\\n# Text Models\\n\\nCopy page\\n\\nQuery, track and manage inference for text models\\n\\nCopy page\\n\\nNew to Fireworks? Start with the [Serverless Quickstart](/getting-started/quickstart) for a step-by-step guide to making your first API call.\\n\\nFireworks provides fast, cost-effective access to leading open-source text models through OpenAI-compatible APIs. Query models via serverless inference or dedicated deployments using the chat completions API (recommended), completions API, or responses API. [Browse 100+ available models \u2192](https://fireworks.ai/models)\\n\\n##\\n\\n\\u200b\\n\\nChat Completions API\\n\\n  * Python\\n\\n  * JavaScript\\n\\n  * curl\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nimport os\\nfrom openai import OpenAI\\n\\nclient = OpenAI(\\n    api_key=os.environ.get(\"FIREWORKS_API_KEY\"),\\n    base_url=\"https://api.fireworks.ai/inference/v1\"\\n)\\n\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\\n        \"role\": \"user\",\\n        \"content\": \"Explain quantum computing in simple terms\"\\n    }]\\n)\\n\\nprint(response.choices[0].message.content)\\n\\n```\\n\\nMost models automatically format your messages with the correct template. To verify the exact prompt used, enable the `echo` parameter.\\n\\n##\\n\\n\\u200b\\n\\nAlternative query methods\\n\\nFireworks also supports [Completions API](/guides/completions-api) and [Responses API](/guides/response-api).\\n\\n##\\n\\n\\u200b\\n\\nQuerying dedicated deployments\\n\\nFor consistent performance, guaranteed capacity, or higher throughput, you can query [on-demand deployments](/guides/ondemand-deployments) instead of serverless models. Deployments use the same APIs with a deployment-specific model identifier:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\n<MODEL_NAME>#<DEPLOYMENT_NAME>\\n\\n```\\n\\nFor example:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1#accounts/<ACCOUNT_ID>/deployments/<DEPLOYMENT_ID>\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}]\\n)\\n\\n```\\n\\n##\\n\\n\\u200b\\n\\nCommon patterns\\n\\n###\\n\\n\\u200b\\n\\nMulti-turn conversations\\n\\nMaintain conversation history by including all previous messages:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nmessages = [\\n    {\"role\": \"system\", \"content\": \"You are a helpful assistant.\"},\\n    {\"role\": \"user\", \"content\": \"What\\'s the capital of France?\"},\\n    {\"role\": \"assistant\", \"content\": \"The capital of France is Paris.\"},\\n    {\"role\": \"user\", \"content\": \"What\\'s its population?\"}\\n]\\n\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=messages\\n)\\n\\nprint(response.choices[0].message.content)\\n\\n```\\n\\nThe model uses the full conversation history to provide contextually relevant responses.\\n\\n###\\n\\n\\u200b\\n\\nSystem prompts\\n\\nOverride the default system prompt by setting the first message with `role: \"system\"`:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nmessages = [\\n    {\"role\": \"system\", \"content\": \"You are a helpful Python expert who provides concise code examples.\"},\\n    {\"role\": \"user\", \"content\": \"How do I read a CSV file?\"}\\n]\\n\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=messages\\n)\\n\\n```\\n\\nTo completely omit the system prompt, set the first message\u2019s `content` to an empty string.\\n\\n###\\n\\n\\u200b\\n\\nStreaming responses\\n\\nStream tokens as they\u2019re generated for real time, interactive UX. Covered in detail in the [Serverless Quickstart](/getting-started/quickstart#streaming-responses).\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nstream = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Tell me a story\"}],\\n    stream=True\\n)\\n\\nfor chunk in stream:\\n    if chunk.choices[0].delta.content:\\n        print(chunk.choices[0].delta.content, end=\"\")\\n\\n```\\n\\n**Aborting streams:** Close the connection to stop generation and avoid billing for ungenerated tokens:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nfor chunk in stream:\\n    print(chunk.choices[0].delta.content, end=\"\")\\n    if some_condition:\\n        stream.close()\\n        break\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nAsync requests\\n\\nUse async clients to make multiple concurrent requests for better throughput:\\n\\n  * Python\\n\\n  * JavaScript\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nimport asyncio\\nfrom openai import AsyncOpenAI\\n\\nclient = AsyncOpenAI(\\n    api_key=os.environ.get(\"FIREWORKS_API_KEY\"),\\n    base_url=\"https://api.fireworks.ai/inference/v1\"\\n)\\n\\nasync def main():\\n    response = await client.chat.completions.create(\\n        model=\"accounts/fireworks/models/deepseek-v3p1\",\\n        messages=[{\"role\": \"user\", \"content\": \"Hello\"}]\\n    )\\n    print(response.choices[0].message.content)\\n\\nasyncio.run(main())\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nUsage & performance tracking\\n\\nEvery response includes token usage information and performance metrics for debugging and observability. For aggregate metrics over time, see the [usage dashboard](https://app.fireworks.ai/account/usage). **Token usage** (prompt, completion, total tokens) is included in the response body for all requests. **Performance metrics** (latency, time-to-first-token, etc.) are included in response headers for non-streaming requests. For streaming requests, use the `perf_metrics_in_response` parameter to include all metrics in the response body.\\n\\n  * Non-streaming\\n\\n  * Streaming (usage only)\\n\\n  * Streaming (with performance metrics)\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}]\\n)\\n\\n# Token usage (always included)\\nprint(response.usage.prompt_tokens)      # Tokens in your prompt\\nprint(response.usage.completion_tokens)  # Tokens generated\\nprint(response.usage.total_tokens)       # Total tokens billed\\n\\n# Performance metrics are in response headers:\\n# fireworks-prompt-tokens, fireworks-server-time-to-first-token, etc.\\n\\n```\\n\\nUsage information is automatically included in the final chunk for streaming responses (the chunk with `finish_reason` set). This is a Fireworks extension - OpenAI SDK doesn\u2019t return usage for streaming by default.\\n\\nFor all available metrics and details, see the [API reference documentation](/api-reference/post-chatcompletions#response-perf_metrics).\\n\\nIf you encounter errors during inference, see [Inference Error Codes](/guides/inference-error-codes) for common issues and resolutions.\\n\\n##\\n\\n\\u200b\\n\\nAdvanced capabilities\\n\\nExtend text models with additional features for structured outputs, tool integration, and performance optimization:\\n\\n## [Tool callingConnect models to external tools and APIs with type-safe parameters](/guides/function-calling)## [Structured outputsEnforce JSON schemas for reliable data extraction](/structured-responses/structured-response-formatting)## [Responses APIMulti-step reasoning for complex problem-solving](/guides/response-api)## [Predicted outputsSpeed up edits by predicting unchanged sections](/guides/predicted-outputs)## [Prompt cachingCache common prompts to reduce latency and cost](/guides/prompt-caching)## [Batch inferenceProcess large volumes of requests asynchronously](/guides/batch-inference)\\n\\n##\\n\\n\\u200b\\n\\nConfiguration & debugging\\n\\nSampling parameters\\n\\nControl how the model generates text. Fireworks automatically uses recommended sampling parameters from each model\u2019s HuggingFace `generation_config.json` when you don\u2019t specify them explicitly, ensuring optimal performance out-of-the-box.We pull `temperature`, `top_k`, `top_p`, `min_p`, and `typical_p` from the model\u2019s configuration when not explicitly provided.\\n\\n###\\n\\n\\u200b\\n\\nTemperature\\n\\nAdjust randomness (0 = deterministic, higher = more creative):\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Write a poem\"}],\\n    temperature=0.7  # Override model default\\n)\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nMax tokens\\n\\nControl the maximum number of tokens in the generated completion:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nmax_tokens=100  # Generate at most 100 tokens\\n\\n```\\n\\n**Important notes:**\\n\\n  * Default value is 2048 tokens if not specified\\n  * Most models support up to their full context window (e.g., 128K for DeepSeek R1)\\n  * When the limit is reached, you\u2019ll see `\"finish_reason\": \"length\"` in the response\\n\\nSet `max_tokens` appropriately for your use case to avoid truncated responses. Check the model\u2019s context window in the [Model Library](https://fireworks.ai/models).\\n\\n###\\n\\n\\u200b\\n\\nTop-p (nucleus sampling)\\n\\nConsider only the most probable tokens summing to `top_p` probability mass:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\ntop_p=0.9  # Consider top 90% probability mass\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nTop-k\\n\\nConsider only the k most probable tokens:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\ntop_k=50  # Consider top 50 tokens\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nMin-p\\n\\nExclude tokens below a probability threshold:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nmin_p=0.05  # Exclude tokens with <5% probability\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nTypical-p\\n\\nUse typical sampling to select tokens with probability close to the entropy of the distribution:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\ntypical_p=0.95  # Consider tokens with typical probability\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nRepetition penalties\\n\\nReduce repetitive text with `frequency_penalty`, `presence_penalty`, or `repetition_penalty`:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nfrequency_penalty=0.5,   # Penalize frequent tokens (OpenAI compatible)\\npresence_penalty=0.5,    # Penalize any repeated token (OpenAI compatible)\\nrepetition_penalty=1.1   # Exponential penalty from prompt + output\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nSampling options header\\n\\nThe `fireworks-sampling-options` header contains the actual default sampling parameters used for the model, including values from the model\u2019s HuggingFace `generation_config.json`:\\n\\n  * Python\\n\\n  * JavaScript\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nimport os\\nfrom openai import OpenAI\\n\\nclient = OpenAI(\\n    api_key=os.environ.get(\"FIREWORKS_API_KEY\"),\\n    base_url=\"https://api.fireworks.ai/inference/v1\"\\n)\\n\\nresponse = client.chat.completions.with_raw_response.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}]\\n)\\n\\n# Access headers from the raw response\\nsampling_options = response.headers.get(\\'fireworks-sampling-options\\')\\nprint(sampling_options)  # e.g., \\'{\"temperature\": 0.7, \"top_p\": 0.9}\\'\\n\\ncompletion = response.parse()  # get the parsed response object\\nprint(completion.choices[0].message.content)\\n\\n```\\n\\nSee the [API reference](/api-reference/post-chatcompletions) for detailed parameter descriptions.\\n\\nMultiple generations\\n\\nGenerate multiple completions in one request:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Tell me a joke\"}],\\n    n=3  # Generate 3 different jokes\\n)\\n\\nfor choice in response.choices:\\n    print(choice.message.content)\\n\\n```\\n\\nToken probabilities (logprobs)\\n\\nInspect token probabilities for debugging or analysis:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    logprobs=True,\\n    top_logprobs=5  # Show top 5 alternatives per token\\n)\\n\\nfor content in response.choices[0].logprobs.content:\\n    print(f\"Token: {content.token}, Logprob: {content.logprob}\")\\n\\n```\\n\\nPrompt inspection (echo & raw_output)\\n\\nVerify how your prompt was formatted:**Echo:** Return the prompt along with the generation:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    echo=True\\n)\\n\\n```\\n\\n**Raw output:** See raw token IDs and prompt fragments:\\n\\nExperimental API - may change without notice.\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    raw_output=True\\n)\\n\\nprint(response.raw_output.prompt_token_ids)  # Token IDs\\nprint(response.raw_output.completion)        # Raw completion\\n\\n```\\n\\nIgnore EOS token\\n\\nForce generation to continue past the end-of-sequence token (useful for benchmarking):\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    ignore_eos=True,\\n    max_tokens=100  # Will always generate exactly 100 tokens\\n)\\n\\n```\\n\\nOutput quality may degrade when ignoring EOS. This API is experimental and should not be relied upon for production use cases.\\n\\nLogit bias\\n\\nModify token probabilities to encourage or discourage specific tokens:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    logit_bias={\\n        123: 10.0,   # Strongly encourage token ID 123\\n        456: -50.0   # Strongly discourage token ID 456\\n    }\\n)\\n\\n```\\n\\nMirostat sampling\\n\\nControl perplexity dynamically using the [Mirostat algorithm](https://arxiv.org/abs/2007.14966):\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nresponse = client.chat.completions.create(\\n    model=\"accounts/fireworks/models/deepseek-v3p1\",\\n    messages=[{\"role\": \"user\", \"content\": \"Hello\"}],\\n    mirostat_target=5.0,  # Target perplexity\\n    mirostat_lr=0.1       # Learning rate for adjustments\\n)\\n\\n```\\n\\n##\\n\\n\\u200b\\n\\nUnderstanding tokens\\n\\nLanguage models process text in chunks called **tokens**. In English, a token can be as short as one character or as long as one word. Different model families use different **tokenizers** , so the same text may translate to different token counts depending on the model. **Why tokens matter:**\\n\\n  * Models have maximum context lengths measured in tokens\\n  * Pricing is based on token usage (prompt + completion)\\n  * Token count affects response time\\n\\nFor Llama models, use [this tokenizer tool](https://belladoreai.github.io/llama-tokenizer-js/example-demo/build/) to estimate token counts. Actual usage is returned in the `usage` field of every API response.\\n\\n##\\n\\n\\u200b\\n\\nOpenAI SDK Migration\\n\\nOpenAI SDK compatibility notes\\n\\nFireworks provides an OpenAI-compatible API, making migration straightforward. However, there are some minor differences to be aware of:\\n\\n###\\n\\n\\u200b\\n\\nBehavioral differences\\n\\n**`stop` parameter:**\\n\\n  * **Fireworks** : Returns text including the stop word\\n  * **OpenAI** : Omits the stop word\\n  * _You can easily truncate it client-side if needed_\\n\\n**`max_tokens` with context limits:**\\n\\n  * **Fireworks** : Automatically adjusts `max_tokens` lower if `prompt + max_tokens` exceeds the model\u2019s context window\\n  * **OpenAI** : Returns an invalid request error\\n  * _Control this behavior with the`context_length_exceeded_behavior` parameter_\\n\\n**Streaming usage stats:**\\n\\n  * **Fireworks** : Returns `usage` field in the final chunk (where `finish_reason` is set) for both streaming and non-streaming\\n  * **OpenAI** : Only returns usage for non-streaming responses\\n\\nExample accessing streaming usage:\\n\\nCopy\\n\\nAsk AI\\n\\n```\\nfor chunk in client.chat.completions.create(stream=True, ...):\\n    if chunk.usage:  # Available in final chunk\\n        print(f\"Tokens: {chunk.usage.total_tokens}\")\\n\\n```\\n\\n###\\n\\n\\u200b\\n\\nUnsupported parameters\\n\\nThe following OpenAI parameters are not yet supported:\\n\\n  * `presence_penalty`\\n  * `frequency_penalty`\\n  * `best_of` (use `n` instead)\\n  * `logit_bias`\\n  * `functions` (deprecated - use [Tool Calling](/guides/function-calling) with the `tools` parameter instead)\\n\\nHave a use case requiring one of these? [Join our Discord](https://discord.gg/fireworks-ai) to discuss.\\n\\n##\\n\\n\\u200b\\n\\nNext steps\\n\\n## [Vision modelsProcess images alongside text](/guides/querying-vision-language-models)## [Audio modelsTranscribe and translate audio](/guides/querying-asr-models)## [EmbeddingsGenerate vector representations for search](/guides/querying-embeddings-models)## [On-demand deploymentsDeploy models on dedicated GPUs](/guides/ondemand-deployments)## [Fine-tuningCustomize models for your use case](/fine-tuning/finetuning-intro)## [Error codesTroubleshoot common inference errors](/guides/inference-error-codes)## [API ReferenceComplete API documentation](/api-reference/post-chatcompletions)\\n\\nWas this page helpful?\\n\\nYesNo\\n\\n[Exporting MetricsPrevious](/deployments/exporting-metrics)[Vision ModelsNext](/guides/querying-vision-language-models)\\n\\n\u2318I\\n\\n[Fireworks AI Docs home page![light logo](https://mintcdn.com/firew<TRUNCATED>"
}
```

</details>

I don't see specific documentation for `kimi-k2-thinking` model parameters in that page. Let me search for more specific information about thinking models and prefilling:Let me search more specifically for prefilling and JSON formatting with thinking models:Based on my search, I don't see specific documentation about prefilling or thinking mode parameters for the `kimi-k2-thinking` model on Fireworks. However, here's what I found:

[*](https://docs.fireworks.ai/api-reference/post-chatcompletions "Create Chat Completion - Fireworks AI Docs") [*](https://docs.fireworks.ai/api-reference/post-chatcompletions "Create Chat Completion - Fireworks AI Docs") For JSON formatting, Fireworks supports `response_format` with `{"type": "json_object"}` to guarantee valid JSON output. [*](https://docs.fireworks.ai/api-reference/post-chatcompletions "Create Chat Completion - Fireworks AI Docs") You should also instruct the model to produce JSON via a system or user message, otherwise it may generate whitespace.

For prefilling with `{`, this is typically an Anthropic-specific feature that may not be available in the Fireworks API. The standard approach would be:

```python
response_format={"type": "json_object"}
```

As for "thinking mode" - [*](https://huggingface.co/moonshotai/Kimi-K2-Thinking "moonshotai/Kimi-K2-Thinking · Hugging Face") Kimi K2 Thinking is built as a thinking agent that reasons step-by-step, so the thinking behavior should be built-in. You likely don't need a special parameter to enable it.

Would you like to try using `response_format` for JSON output instead of prefilling?

Yes how to change the generator code?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You'll need to modify the `generate` method in your `Generator` class to add the `response_format` parameter. Here's the key change:

```python
response = self.llm_client.chat.completions.create(
    **formatted_prompt['messages'],
    response_format={"type": "json_object"}
)
```

However, I notice your current code is calling `formatted_prompt['messages']` directly - can you show me what your `prompt_template.build()` method returns? That will help me give you the exact syntax you need.

In [ ]:
loaded_prompt = braintrust.load_prompt(project="wattbot", slug="wattbot-prompt-v1")

In [ ]:
qa.iloc[0]

id                                                                   q003
question                What is the name of the benchmark suite presen...
answer                                            The ML.ENERGY Benchmark
answer_value                                          ML.ENERGY Benchmark
answer_unit                                                      is_blank
ref_id                                                      ['chung2025']
ref_url                              ['https://arxiv.org/pdf/2505.06371']
supporting_materials    We present the ML.ENERGY Benchmark, a benchmar...
explanation                                                         Quote
Name: 0, dtype: object

In [ ]:
query_text = 'What is the name of the benchmark suite presented in a recent paper for measuring inference energy consumption?'
retrieved_chunks = Retriever(embedder, qs).retrieve(query_text, limit=10)
generator = Generator(llm, loaded_prompt)
generator.generate(query_text, retrieved_chunks)

{'answer': 'ML.ENERGY Benchmark',
 'answer_value': 'ML.ENERGY Benchmark',
 'answer_unit': 'is_blank',
 'ref_id': ['chung2025'],
 'ref_url': ['https://arxiv.org/pdf/2505.06371'],
 'supporting_materials': 'We present the ML.ENERGY Benchmark, a benchmark suite and tool for measuring inference energy consumption under realistic service environments',
 'explanation': 'Quote'}

## RAG

In [ ]:
class RAG:
    def __init__(self, retriever, generator):
        fc.store_attr()
    
    def answer_question(self, query, limit=5):
        retrieved_chunks = self.retriever.retrieve(query, limit=limit)
        return self.generator.generate(query, retrieved_chunks), retrieved_chunks
    
    def predict_dataset(self, df, limit=5, n_workers=4):
        def process_row(idx_r):
            idx, r = idx_r
            answer, retrieved_chunks = self.answer_question(r['question'], limit=limit)
            if not isinstance(answer, dict): print(idx)
            return {
                'id': r['id'],
                'question': r['question'],
                'answer': answer['answer'],
                'answer_value': answer['answer_value'],
                'answer_unit': answer['answer_unit'],
                'ref_id': answer['ref_id'],
                'ref_url': answer['ref_url'],
                'supporting_materials': answer['supporting_materials'],
                'explanation': answer['explanation']
            }
    
        results = fc.parallel(process_row, df.iterrows(), threadpool=True, n_workers=n_workers)
        return pd.DataFrame(list(results))

In [ ]:
rag = RAG(retriever, generator)
pred_answer, retrieved_chunks = rag.answer_question(query_text, limit=10); pred_answer

{'answer': 'ML.ENERGY Benchmark',
 'answer_value': 'ML.ENERGY Benchmark',
 'answer_unit': 'is_blank',
 'ref_id': ['chung2025'],
 'ref_url': ['https://arxiv.org/pdf/2505.06371'],
 'supporting_materials': 'The ML.ENERGY Benchmark: Toward Automated Inference Energy Measurement and Optimization',
 'explanation': 'Quote'}

## Evaluation

In [ ]:
def is_numeric(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

In [ ]:
def score_answer_value(predicted, expected):
    if expected == "is_blank":
        return 1.0 if predicted == "is_blank" else 0.0
    
    if isinstance(expected, str) and expected.startswith('['): expected = ast.literal_eval(expected)
    
    if is_numeric(expected) and is_numeric(predicted):
        pred_num, exp_num = map(float, (predicted, expected))
        return 1.0 if abs(pred_num - exp_num) <= abs(exp_num * 0.001) else 0.0
    else:
        return 1.0 if str(predicted).strip().lower() == str(expected).strip().lower() else 0.0

In [ ]:
score_answer_value([80,90], "[80,90]") 

1.0

I want to change the answer_value to evaluate the ranges properly

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

To handle range evaluation properly, you need to detect when the expected value is a range (like `[80,90]`) and check if the predicted value falls within that range.

Here are a few approaches:

1. Parse ranges and check if predicted value is within bounds
2. Handle both list formats `[80,90]` and string formats `"[80,90]"`
3. Consider whether predicted should be a single value in the range or also a range

What format is your predicted value in - is it a single number, a list, or a string? And should it match the exact range or just fall within it?

The answer value will be a list but the dataframe can have string of list

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Got it! So you need to:

1. Parse the string representation of a list (like `"[80,90]"`) into an actual list
2. Compare two lists for equality

You can use `ast.literal_eval()` to safely parse the string into a list:

```python
import ast
parsed = ast.literal_eval("[80,90]")  # Returns [80, 90]
```

Then compare the predicted list with the parsed expected list. Would you like to update the `score_answer_value` function to handle this, or would you prefer to try it yourself first?

Please update it

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's the updated function to handle list comparisons:

```python
def score_answer_value(predicted, expected):
    if expected == "is_blank":
        return 1.0 if predicted == "is_blank" else 0.0
    
    # Handle list/range comparisons
    if isinstance(expected, str) and expected.startswith('['):
        try:
            expected = ast.literal_eval(expected)
        except (ValueError, SyntaxError):
            pass
    
    if isinstance(expected, list) and isinstance(predicted, list):
        return 1.0 if predicted == expected else 0.0
        
    if is_numeric(expected) and is_numeric(predicted):
        pred_num, exp_num = map(float, (predicted, expected))
        return 1.0 if abs(pred_num - exp_num) <= abs(exp_num * 0.001) else 0.0
    else:
        return 1.0 if str(predicted).strip().lower() == str(expected).strip().lower() else 0.0
```

This adds list handling before the numeric check. Try it with your test case and let me know if it works!

In [ ]:
score_answer_value(pred_answer['answer_value'], qa.iloc[0]['answer_value'])

1.0

In [ ]:
def score_ref_id(predicted, expected):
    if expected == "is_blank":
        return 1.0 if predicted == "is_blank" else 0.0
        
    if isinstance(expected, str) and expected.startswith('['): expected = ast.literal_eval(expected)
        
    pred_set = set(predicted) if isinstance(predicted, list) else set([predicted])
    exp_set = set(expected) if isinstance(expected, list) else set([expected])
        
    intersection = len(pred_set.intersection(exp_set))
    union = len(pred_set.union(exp_set))
        
    return intersection / union if union > 0 else 0.0

In [ ]:
def score_is_na(predicted_answer, expected_answer):
    expected_is_na = expected_answer['answer_value'] == 'is_blank'
    predicted_is_na = predicted_answer['answer_value'] == 'is_blank'
    return 1.0 if expected_is_na == predicted_is_na else 0.0

In [ ]:
def calculate_wattbot_score(predicted_answer, expected_row):
    if 'answer_value' not in predicted_answer: print(expected_row['id'], predicted_answer)
    answer_score = score_answer_value(predicted_answer['answer_value'], expected_row['answer_value'])
    ref_score = score_ref_id(predicted_answer['ref_id'], expected_row['ref_id'])
    na_score = score_is_na(predicted_answer, expected_row)
    score = 0.75 * answer_score + 0.15 * ref_score + 0.10 * na_score
    return fc.NS(score=score, answer_score=answer_score, ref_score=ref_score, na_score=na_score)

In [ ]:
calculate_wattbot_score(pred_answer, qa.iloc[0])

namespace(score=1.0, answer_score=1.0, ref_score=1.0, na_score=1.0)

In [ ]:
class Evaluator:
    def __init__(self, rag):
        fc.store_attr()
        self.results = []
    
    def evaluate_dataset(self, df, limit=5):
        experiment = braintrust.init(project="wattbot", experiment="evaluation")
        def evaluate_row(idx_r):
            idx, r = idx_r
            predicted_answer, retrieved_chunks = self.rag.answer_question(r['question'], limit=limit)
            wattbot_score = calculate_wattbot_score(predicted_answer, r)
            context = self.rag.generator.create_context(retrieved_chunks)
            prompt = self.rag.generator.format_prompt(r['question'], context)
            experiment.log(input=r['question'], output=predicted_answer, expected=r, scores=wattbot_score.__dict__, metadata = {'context': context, 'prompt': prompt})
            return fc.NS(qid=r['id'], q=r['question'], predicted=predicted_answer, expected=r, **wattbot_score.__dict__, context=context)
        
        self.results = fc.parallel(evaluate_row, df.iterrows(), threadpool=True)
        return sum(r.score for r in self.results) / len(self.results)
    
    def get_worst_predictions(self, n=5):
        sorted_results = sorted(self.results, key=lambda r: r.score)
        return sorted_results[:n]

How can I have the context and prompt logged into braintrust respecting the newlines?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The newlines in your context and prompt strings might not be preserved when logged to Braintrust. To ensure they're displayed properly, you can try:

1. **Store as a list instead of a single string** - Break the context into separate chunks
2. **Use explicit newline encoding** - Though this usually isn't necessary
3. **Check Braintrust's UI settings** - Some UIs have options to preserve formatting

The simplest approach is to log the context as a structured object:

```python
metadata = {
    'context_chunks': [chunk.payload for chunk in retrieved_chunks],
    'prompt': prompt
}
```

Would you like to try this approach, or would you prefer to keep it as a single formatted string and investigate Braintrust's display options?

In braintrust for some examples it is respecting the new line and for some it does not. Is there an issue on our code?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

That's interesting - inconsistent newline handling suggests the issue might be with how the data is being formatted before logging.

A few things to check:

1. Are the newlines actually in the strings (`\n`) or are they literal line breaks?
2. Is the `context` or `prompt` being processed differently in some cases (e.g., through JSON serialization)?
3. Are you logging strings vs dict values differently?

Could you show me an example of one that works and one that doesn't? Or check if the `create_context` method is consistently returning strings with `\n` characters?

Yes please

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The core issue is that `self.rag.retrieved_chunks` is shared state that gets overwritten by concurrent threads. Here are two solutions:

**Option 1: Make RAG return chunks along with the answer**

Modify your `RAG.answer_question` method to return both the answer and the chunks:

```python
def answer_question(self, query, limit=5):
    retrieved_chunks = self.retriever.retrieve(query, limit=limit)
    answer = self.generator.generate(query, retrieved_chunks)
    return answer, retrieved_chunks
```

Then update your evaluator to use the returned chunks instead of the shared state.

**Option 2: Remove threading**

Change `threadpool=True` to `threadpool=False` in your `fc.parallel` call. This makes execution sequential, avoiding the race condition entirely.

Which approach would you prefer? Option 1 is better if you want to keep parallel execution for speed, while Option 2 is simpler if speed isn't critical.

Do you see any other race condition problem with the code?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! Looking at your code more carefully, there's another race condition in the `Generator` class and the `braintrust.init` call:

1. **Braintrust experiment object**: You're creating `experiment = braintrust.init(...)` once, but then multiple threads are calling `experiment.log()` simultaneously. This could cause issues depending on whether Braintrust's logging is thread-safe.

2. **LLM client state**: Your `Generator` uses `self.llm_client`, and if multiple threads call `generate()` at the same time, they're all sharing the same client connection. This might work depending on the client implementation, but could cause problems.

Do you want to explore solutions for these, or would you like to first check if there are other issues you've noticed?

In [ ]:
evals = Evaluator(rag)
evals.evaluate_dataset(df=qa, limit=10)

Skipping git metadata. This is likely because the repository has not been published to a remote yet. Remote named 'origin' didn't exist


0.7838414634146342

Why some of the outputs are empty? This is for the kimi k2 thinking model
whereas the deepseek model gives proper output

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The empty outputs (`{}`) from the Kimi K2 Thinking model suggest it's not returning valid JSON in some cases. This could be due to:

1. **Thinking tokens interfering with JSON** - The model might be outputting reasoning/thinking text before the JSON, breaking the `json.loads()` parsing
2. **Response format not properly enforced** - The `response_format={"type": "json_object"}` might not be working as expected with this model
3. **Model-specific behavior** - Thinking models sometimes structure their output differently

To debug this, you could:

1. Print the raw response content before parsing to see what the model is actually returning
2. Try adding error handling to catch and log failed parses
3. Check if the prompt needs to be more explicit about JSON-only output for thinking models

Would you like to add some debugging to see what the raw responses look like?

In [ ]:
tst_ans = rag.predict_dataset(tst)

In [ ]:
tst_ans.isnull().sum()

id                      0
question                0
answer                  0
answer_value            0
answer_unit             0
ref_id                  0
ref_url                 0
supporting_materials    0
explanation             0
dtype: int64

In [ ]:
tst_ans.to_csv(data_path/'tst_ans_v5.csv', index=False)

 What are other ways to improve this model? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! Here are several ways you could improve your RAG model's performance:

**Retrieval improvements:**
- Experiment with different chunk sizes and overlap (your current `chunk_size=1500, step=1400`)
- Try different embedding models or ensemble multiple embeddings
- Adjust the hybrid search fusion weights between dense and sparse
- Increase the number of retrieved chunks and use reranking

**Generation improvements:**
- Experiment with different LLMs or prompt engineering
- Add few-shot examples to your prompt template
- Implement answer verification or self-consistency checks

**Evaluation-driven improvements:**
- Analyze your worst predictions (you already have `get_worst_predictions`)
- Look for patterns in what types of questions fail

Which area interests you most, or would you like to start by examining your worst predictions to see where the model struggles?

In [ ]:
evals.get_worst_predictions()

[namespace(qid='q280',
           q='Given the total pre-training GPU hours and the number of GPUs used, estimate the total wall-clock time in days required to pre-train the JetMoE-8B model.',
           predicted={'answer': 'Unanswerable from the provided context',
                      'answer_value': 'is_blank',
                      'answer_unit': 'is_blank',
                      'ref_id': ['is_blank'],
                      'ref_url': ['is_blank'],
                      'supporting_materials': 'is_blank',
                      'explanation': 'is_blank'},
           expected=id                                                                   q280
                    question                Given the total pre-training GPU hours and the...
                    answer                                                           ~13 days
                    answer_value                                                           13
                    answer_unit                          

What do you think where the RAG is going wrong?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Looking at your worst predictions, I can see a few key patterns where the RAG is failing:

**1. Missing the correct source document entirely**
- For q280 about JetMoE-8B, none of the retrieved chunks are from `shen2024` (the expected reference)
- For q170 and q262, the chunks are from related papers (dodge2022, luccioni2023) but not from `strubell2019` which has the specific answer

**2. Right document, wrong chunk**
- Even when the document is retrieved (like q078), the specific numerical detail might not be in the chunk, or it's buried in a way that's hard for the LLM to extract

**3. Semantic vs. keyword mismatch**
- Questions asking for very specific numbers/calculations might not embed similarly to the text containing those numbers

**4. Answer format issues**
- For q078, the model found the right info but reported it inverted (10-50 responses per bottle instead of 0.02-0.1 bottles per response)

Would you like to dig deeper into any of these patterns, or try some experiments to improve retrieval for these specific failure cases?

Sure

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start by investigating the retrieval issue more closely. 

For the failed questions, would you like to:

1. **Search for specific text** from the expected answers in your document chunks to see if they're actually there
2. **Compare embeddings** between the query and the correct chunk to understand why it ranked lower
3. **Try different retrieval parameters** (like increasing the limit or adjusting hybrid search weights)

Which approach would you like to explore first?

Will chaning the embedding model improve the score?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

It could, but it depends on what's causing the failures. Let me break this down:

**When a different embedding model might help:**
- If your current model (nomic-embed-text) isn't capturing domain-specific terminology well
- If it's not good at distinguishing between similar technical concepts
- If there's a mismatch between how queries and documents are embedded

**When it might NOT help much:**
- If the problem is that the correct chunks simply don't contain the specific facts (chunking issue)
- If the information is there but the LLM isn't extracting it correctly (generation issue)
- If your hybrid search weights need tuning

**To test this hypothesis**, you could:
1. Check if the correct chunks are even in your top-10 or top-20 results
2. Try a domain-specific embedding model (like one trained on scientific papers)
3. Compare results with your current model

Do you want to first check whether the right chunks are being retrieved at all (just ranked lower), or would you prefer to try a different embedding model right away?

I think the worst predictions are already there, what do you think?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're absolutely right! You already have the worst predictions with `evals.get_worst_predictions()`. 

Looking at those results, the key diagnostic question is: **Are the correct chunks being retrieved at all, just ranked poorly?**

You could check this by:
- Looking at whether the expected `ref_id` appears anywhere in the top 10-20 retrieved chunks
- Searching for the specific answer text in your chunks to see if it exists

For example, for q280 about JetMoE-8B, you could search all chunks for "JetMoE" or "30,000" to see if that information is even in your corpus.

Would you like to write a quick check to see if the expected document IDs appear in the retrieved chunks for your worst predictions?

I want to improve the figure and table intepretation from the pdf

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good idea! Improving figure and table interpretation could help with those questions that reference specific data points.

Here are a few approaches you could take:

1. **Extract tables separately** - Use a specialized tool to parse tables from PDFs and store them with better structure
2. **Add visual context** - Include table/figure captions and surrounding text when chunking
3. **Use multimodal embeddings** - Embed images of tables/figures along with text
4. **Structured extraction** - Convert tables to structured formats (CSV, JSON) before embedding

Which approach interests you most, or would you like to explore what's currently happening with tables in your chunks?

Also the ref_id has square bracket even though the answer is is_blank for example ["is_blank"]

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good catch! That's a data consistency issue. The `ref_id` should be `"is_blank"` (a string) when the answer is blank, not `["is_blank"]` (a list containing the string).

This could be happening in:
1. Your prompt template - if it's instructing the model to always return a list
2. Your data preprocessing - if you're wrapping everything in lists
3. The LLM's output - if it's being too literal about the JSON schema

Would you like to check your prompt template to see how `ref_id` is defined, or would you prefer to add post-processing to normalize this?

Lets address the table and image extraction. I think we are missing a great deal of information there.
I want to generate the embedding and the text from the table/image also have the image/table part of the payload if possible for further debugging

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great plan! Extracting tables and images separately will likely improve your results significantly.

Here's the approach:

1. **Extract tables/images from PDFs** - Use a library like `pdfplumber` for tables or a multimodal model for images
2. **Convert to text** - Tables → markdown/CSV format, Images → captions/descriptions
3. **Store in chunks** - Include the extracted text plus metadata (page number, type: "table"/"image")
4. **Optionally store raw data** - Keep the image bytes or table structure in payload for debugging

For your use case, would you like to:
- Start with just tables (simpler, likely covers most of your numerical data)?
- Or handle both tables and images from the beginning?

Lets start with table

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! For table extraction from PDFs, you have a few good options:

1. **pdfplumber** - Good for structured tables, extracts to list of dicts
2. **camelot-py** - Specialized for tables, handles complex layouts
3. **tabula-py** - Java-based, exports to pandas DataFrames
4. **Multimodal LLM** - Use vision models to "read" tables from PDF images

For your wattbot data with research papers, I'd recommend starting with **pdfplumber** since it's already available and works well with academic PDFs.

Would you like to try extracting tables from one of your PDFs to see what we get, or would you prefer me to show you a small code example first?

I also want to see why my rd.read_pdf does not do a good job in extracting table

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good idea to investigate! `contextkit.read_pdf` likely uses a basic text extraction method that doesn't handle table structure well - it probably just extracts text in reading order, losing the column/row relationships.

To see what's happening, you could:

1. Pick a PDF with a known table (like one of your papers with data tables)
2. Extract it with `rd.read_pdf()` and look at the output
3. Compare with what a table-specific tool extracts

Want to try this with one of your documents to see the difference?

I have a doc id strubell2019

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Let's look at what `rd.read_pdf` extracts from that document. Try this:

```python
content = rd.read_pdf(data_path/'strubell2019.pdf')
```

Then search for a section that should contain a table (like "Table 1" or similar) and print a chunk around it to see how the table data looks.

Would you like to do this, or should I help you with the code?

In [ ]:
content = rd.read_pdf(data_path/'strubell2019.pdf')

In [ ]:
content

'arXiv:1906.02243v1  [cs.CL]  5 Jun 2019\nEnergy and Policy Considerations for Deep Learning in NLP\nEmma Strubell Ananya Ganesh Andrew McCallum\nCollege of Information and Computer Sciences\nUniversity of Massachusetts Amherst\n{strubell, aganesh, mccallum}@cs.umass.edu\nAbstract\nRecent progress in hardware and methodol-\nogy for training neural networks has ushered\nin a new generation of large networks trained\non abundant data. These models have ob-\ntained notable gains in accuracy across many\nNLP tasks. However, these accuracy improve-\nments depend on the availability of exception-\nally large computational resources that neces-\nsitate similarly substantial energy consump-\ntion. As a result these models are costly to\ntrain and develop, both ﬁnancially, due to the\ncost of hardware and electricity or cloud com-\npute time, and environmentally, due to the car-\nbon footprint required to fuel modern tensor\nprocessing hardware. In this paper we bring\nthis issue to the attenti

It is able to extract information from the table

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

You're right! Looking at the output, `rd.read_pdf` does extract the table content from strubell2019. For example, I can see Table 1 with the CO2 emissions data and Table 2 with the energy sources.

However, the structure is lost - it's just linear text. So questions like "How many days of CO₂ emissions from an average American life are equivalent to training BERT base?" require the model to parse:

```
Consumption CO 2e (lbs)
...
American life, avg, 1 year 36,156
...
BERT base ... 1438
```

And then calculate: 1438 / (36,156/365) = 14.4 days

The challenge is that this information is spread across different parts of the text, and the table structure that makes it easy for humans to read is gone.

Would you still like to try extracting tables separately to preserve their structure better, or do you think the current extraction is sufficient?

It will better to try extracting tables separately to preserve the structure better

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good choice! Extracting tables with their structure preserved will definitely help with those numerical questions.

For table extraction, you have a few options:

1. **pdfplumber** - Extract tables as lists of dictionaries, good for structured tables
2. **camelot-py** - Specialized for complex table layouts
3. **PyMuPDF (fitz)** - Can extract tables and also get page images

Which one would you like to try first? I'd suggest starting with **pdfplumber** since it's straightforward and handles most academic paper tables well.

Extracting the table will not make it duplicate of the existing table information that is extracted rd.read_pdf
How can we avoid that?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great point! You don't want to duplicate the same information. Here are a few approaches:

1. **Replace table text with structured version** - When you find a table, remove that text from the main content and replace it with a markdown/structured version
2. **Store tables as separate chunks** - Extract tables separately and mark them with `type: "table"` in metadata, then skip those page regions when doing regular text extraction
3. **Hybrid approach** - Keep both but use metadata to deduplicate during retrieval (e.g., filter by chunk type)

The cleanest approach is probably #2 - extract tables separately with special metadata, then use a table-aware PDF reader that skips table regions.

Which approach sounds best for your use case?

Lets try to use pdf plumber and see if it is able to differentiate then we can use pdf plumber instead of rd.read_pdf

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good plan! Let's test pdfplumber on the strubell2019 PDF to see what it extracts.

First, you'll need to install it if you don't have it:

```python
!pip install pdfplumber
```

Then try extracting from one page to see what tables it finds:

```python
import pdfplumber

with pdfplumber.open(data_path/'strubell2019.pdf') as pdf:
    first_page = pdf.pages[0]
    tables = first_page.extract_tables()
    print(f"Found {len(tables)} tables on page 1")
    if tables:
        print(tables[0])
```

This will show you if pdfplumber can identify and extract tables separately. Want to try this?

In [ ]:
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.6 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.8 MB/s  0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.2 MB/s  0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [pdfminer.six]

   ━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [pdfminer.six]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pdfplumber]


In [ ]:
import pdfplumber

with pdfplumber.open(data_path/'strubell2019.pdf') as pdf:
    first_page = pdf.pages[0]
    tables = first_page.extract_tables()
    print(f"Found {len(tables)} tables on page 1")
    if tables:
        print(tables[0])

Found 0 tables on page 1


It could not find any table

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The table might be on a different page. Try checking a few more pages to see where the tables are:

```python
with pdfplumber.open(data_path/'strubell2019.pdf') as pdf:
    for i, page in enumerate(pdf.pages[:5]):  # Check first 5 pages
        tables = page.extract_tables()
        if tables:
            print(f"Page {i}: Found {len(tables)} table(s)")
```

This will show you which pages have tables that pdfplumber can detect. Want to try this?

In [ ]:
with pdfplumber.open(data_path/'strubell2019.pdf') as pdf:
    for i, page in enumerate(pdf.pages[:5]):  # Check first 5 pages
        tables = page.extract_tables()
        if tables:
            print(f"Page {i}: Found {len(tables)} table(s)")


It was on the first page

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Interesting - so pdfplumber isn't detecting the table on page 1 even though we can see it in the text. This could be because:

1. The table isn't structured with proper borders/lines in the PDF
2. It's formatted as plain text rather than a table object
3. pdfplumber's default settings are too strict

You could try adjusting pdfplumber's table detection settings, or alternatively, try a different tool like **camelot-py** which is better at detecting borderless tables.

Would you like to try camelot, or adjust pdfplumber's settings?

In [ ]:
??rd.read_pdf


```python
def read_pdf(file_path: str # path of PDF file to read
            ) -> str:
    "Reads the text of a PDF with PdfReader"
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        return ' '.join(page.extract_text() for page in reader.pages)
```

**File:** `/usr/local/lib/python3.12/site-packages/contextkit/read.py`

There is a python library marker https://github.com/datalab-to/marker which also extracts tables and images. I want to use it to extract chunks

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great find! Marker is a modern library specifically designed for converting PDFs to markdown while preserving tables, images, and structure. It uses ML models for better extraction.

To use it, you'd need to:

1. Install marker: `!pip install marker-pdf`
2. Use it to convert your PDFs to markdown format
3. Parse the markdown to extract text, tables, and images separately
4. Create chunks with appropriate metadata

Would you like to try installing it and testing on one of your PDFs to see how well it extracts the tables?

It has its own in built functionality to create chunks, lets use that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Marker has built-in chunking functionality which should handle tables and images intelligently.

You'll want to explore Marker's chunking API to see:
1. What metadata it provides for each chunk (like whether it's a table, image, or text)
2. How it handles table formatting
3. What configuration options are available

Would you like to install marker and test it on one PDF first to see what the chunks look like?

Can you please read the repo?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->



<details class='tool-usage-details'>

```json
{
  "id": "toolu_012Avekt16UrcHMDNJjT9saE",
  "call": {
    "function": "read_url",
    "arguments": {
      "url": "https://github.com/datalab-to/marker"
    }
  },
  "result": "'Skip to content\\n\\n## Navigation Menu\\n\\nToggle navigation\\n\\n[ ](/)\\n\\n[ Sign in ](/login?return_to=https%3A%2F%2Fgithub.com%2Fdatalab-to%2Fmarker)\\n\\nAppearance settings\\n\\n  * Platform \\n\\n    * [ GitHub Copilot  Write better code with AI  ](https://github.com/features/copilot)\\n    * [ GitHub Spark  New  Build and deploy intelligent apps  ](https://github.com/features/spark)\\n    * [ GitHub Models  New  Manage and compare prompts  ](https://github.com/features/models)\\n    * [ GitHub Advanced Security  Find and fix vulnerabilities  ](https://github.com/security/advanced-security)\\n    * [ Actions  Automate any workflow  ](https://github.com/features/actions)\\n\\n    * [ Codespaces  Instant dev environments  ](https://github.com/features/codespaces)\\n    * [ Issues  Plan and track work  ](https://github.com/features/issues)\\n    * [ Code Review  Manage code changes  ](https://github.com/features/code-review)\\n    * [ Discussions  Collaborate outside of code  ](https://github.com/features/discussions)\\n    * [ Code Search  Find more, search less  ](https://github.com/features/code-search)\\n\\nExplore\\n\\n    * [ Why GitHub ](https://github.com/why-github)\\n    * [ Documentation  ](https://docs.github.com)\\n    * [ GitHub Skills  ](https://skills.github.com)\\n    * [ Blog  ](https://github.blog)\\n\\nIntegrations\\n\\n    * [ GitHub Marketplace ](https://github.com/marketplace)\\n    * [ MCP Registry ](https://github.com/mcp)\\n\\n[ View all features  ](https://github.com/features)\\n\\n  * Solutions \\n\\nBy company size\\n\\n    * [ Enterprises ](https://github.com/enterprise)\\n    * [ Small and medium teams ](https://github.com/team)\\n    * [ Startups ](https://github.com/enterprise/startups)\\n    * [ Nonprofits ](/solutions/industry/nonprofits)\\n\\nBy use case\\n\\n    * [ App Modernization ](/solutions/use-case/app-modernization)\\n    * [ DevSecOps ](/solutions/use-case/devsecops)\\n    * [ DevOps ](/solutions/use-case/devops)\\n    * [ CI/CD ](/solutions/use-case/ci-cd)\\n    * [ View all use cases ](/solutions/use-case)\\n\\nBy industry\\n\\n    * [ Healthcare ](/solutions/industry/healthcare)\\n    * [ Financial services ](/solutions/industry/financial-services)\\n    * [ Manufacturing ](/solutions/industry/manufacturing)\\n    * [ Government ](/solutions/industry/government)\\n    * [ View all industries ](/solutions/industry)\\n\\n[ View all solutions  ](/solutions)\\n\\n  * Resources \\n\\nTopics\\n\\n    * [ AI ](/resources/articles?topic=ai)\\n    * [ DevOps ](/resources/articles?topic=devops)\\n    * [ Security ](/resources/articles?topic=security)\\n    * [ Software Development ](/resources/articles?topic=software-development)\\n    * [ View all ](/resources/articles)\\n\\nExplore\\n\\n    * [ Learning Pathways  ](https://resources.github.com/learn/pathways)\\n    * [ Events & Webinars ](https://github.com/resources/events)\\n    * [ Ebooks & Whitepapers ](https://github.com/resources/whitepapers)\\n    * [ Customer Stories ](https://github.com/customer-stories)\\n    * [ Partners ](https://github.com/partners)\\n    * [ Executive Insights ](https://github.com/solutions/executive-insights)\\n\\n  * Open Source \\n\\n    * [ GitHub Sponsors  Fund open source developers  ](/sponsors)\\n\\n    * [ The ReadME Project  GitHub community articles  ](https://github.com/readme)\\n\\nRepositories\\n\\n    * [ Topics ](https://github.com/topics)\\n    * [ Trending ](https://github.com/trending)\\n    * [ Collections ](https://github.com/collections)\\n\\n  * Enterprise \\n\\n    * [ Enterprise platform  AI-powered developer platform  ](/enterprise)\\n\\nAvailable add-ons\\n\\n    * [ GitHub Advanced Security  Enterprise-grade security features  ](https://github.com/security/advanced-security)\\n    * [ Copilot for business  Enterprise-grade AI features  ](/features/copilot/copilot-business)\\n    * [ Premium Support  Enterprise-grade 24/7 support  ](/premium-support)\\n\\n  * [Pricing](https://github.com/pricing)\\n\\nSearch or jump to...\\n\\n# Search code, repositories, users, issues, pull requests...\\n\\nSearch\\n\\nClear\\n\\n[Search syntax tips](https://docs.github.com/search-github/github-code-search/understanding-github-code-search-syntax)\\n\\n#  Provide feedback\\n\\nWe read every piece of feedback, and take your input very seriously.\\n\\nInclude my email address so I can be contacted\\n\\nCancel  Submit feedback\\n\\n#  Saved searches\\n\\n## Use saved searches to filter your results more quickly\\n\\nName\\n\\nQuery\\n\\nTo see all available qualifiers, see our [documentation](https://docs.github.com/search-github/github-code-search/understanding-github-code-search-syntax).\\n\\nCancel  Create saved search\\n\\n[ Sign in ](/login?return_to=https%3A%2F%2Fgithub.com%2Fdatalab-to%2Fmarker)\\n\\n[ Sign up ](/signup?ref_cta=Sign+up&ref_loc=header+logged+out&ref_page=%2F%3Cuser-name%3E%2F%3Crepo-name%3E&source=header-repo&source_repo=datalab-to%2Fmarker)\\n\\nAppearance settings\\n\\nResetting focus\\n\\nYou signed in with another tab or window. [Reload]() to refresh your session. You signed out in another tab or window. [Reload]() to refresh your session. You switched accounts on another tab or window. [Reload]() to refresh your session. Dismiss alert\\n\\n{{ message }}\\n\\n[ datalab-to ](/datalab-to) / **[marker](/datalab-to/marker) ** Public\\n\\n  * [ Notifications ](/login?return_to=%2Fdatalab-to%2Fmarker) You must be signed in to change notification settings\\n  * [ Fork 2k ](/login?return_to=%2Fdatalab-to%2Fmarker)\\n  * [ Star  29.9k ](/login?return_to=%2Fdatalab-to%2Fmarker)\\n\\nConvert PDF to markdown + JSON quickly with high accuracy\\n\\n[www.datalab.to](https://www.datalab.to \"https://www.datalab.to\")\\n\\n### License\\n\\n[ GPL-3.0 license ](/datalab-to/marker/blob/master/LICENSE)\\n\\n[ 29.9k stars ](/datalab-to/marker/stargazers) [ 2k forks ](/datalab-to/marker/forks) [ Branches ](/datalab-to/marker/branches) [ Tags ](/datalab-to/marker/tags) [ Activity ](/datalab-to/marker/activity)\\n\\n[ Star  ](/login?return_to=%2Fdatalab-to%2Fmarker)\\n\\n[ Notifications ](/login?return_to=%2Fdatalab-to%2Fmarker) You must be signed in to change notification settings\\n\\n  * [ Code ](/datalab-to/marker)\\n  * [ Issues 298 ](/datalab-to/marker/issues)\\n  * [ Pull requests 46 ](/datalab-to/marker/pulls)\\n  * [ Actions ](/datalab-to/marker/actions)\\n  * [ Projects 0 ](/datalab-to/marker/projects)\\n  * [ Security Uh oh!  There was an error while loading. [Please reload this page](). ](/datalab-to/marker/security)\\n  * [ Insights ](/datalab-to/marker/pulse)\\n\\nAdditional navigation options\\n\\n  * [ Code  ](/datalab-to/marker)\\n  * [ Issues  ](/datalab-to/marker/issues)\\n  * [ Pull requests  ](/datalab-to/marker/pulls)\\n  * [ Actions  ](/datalab-to/marker/actions)\\n  * [ Projects  ](/datalab-to/marker/projects)\\n  * [ Security  ](/datalab-to/marker/security)\\n  * [ Insights  ](/datalab-to/marker/pulse)\\n\\n# datalab-to/marker\\n\\nmaster\\n\\n[Branches](/datalab-to/marker/branches)[Tags](/datalab-to/marker/tags)\\n\\n[](/datalab-to/marker/branches)[](/datalab-to/marker/tags)\\n\\nGo to file\\n\\nCode\\n\\nOpen more actions menu\\n\\n## Folders and files\\n\\nName| Name| Last commit message| Last commit date  \\n---|---|---|---  \\n  \\n## Latest commit\\n\\n## History\\n\\n[1,339 Commits](/datalab-to/marker/commits/master/)[](/datalab-to/marker/commits/master/)  \\n[.github](/datalab-to/marker/tree/master/.github \".github\")| [.github](/datalab-to/marker/tree/master/.github \".github\")|  |   \\n[benchmarks](/datalab-to/marker/tree/master/benchmarks \"benchmarks\")| [benchmarks](/datalab-to/marker/tree/master/benchmarks \"benchmarks\")|  |   \\n[data](/datalab-to/marker/tree/master/data \"data\")| [data](/datalab-to/marker/tree/master/data \"data\")|  |   \\n[examples](/datalab-to/marker/tree/master/examples \"examples\")| [examples](/datalab-to/marker/tree/master/examples \"examples\")|  |   \\n[marker](/datalab-to/marker/tree/master/marker \"marker\")| [marker](/datalab-to/marker/tree/master/marker \"marker\")|  |   \\n[signatures/version1](/datalab-to/marker/tree/master/signatures/version1 \"This path skips through empty directories\")| [signatures/version1](/datalab-to/marker/tree/master/signatures/version1 \"This path skips through empty directories\")|  |   \\n[static/fonts](/datalab-to/marker/tree/master/static/fonts \"This path skips through empty directories\")| [static/fonts](/datalab-to/marker/tree/master/static/fonts \"This path skips through empty directories\")|  |   \\n[tests](/datalab-to/marker/tree/master/tests \"tests\")| [tests](/datalab-to/marker/tree/master/tests \"tests\")|  |   \\n[.gitignore](/datalab-to/marker/blob/master/.gitignore \".gitignore\")| [.gitignore](/datalab-to/marker/blob/master/.gitignore \".gitignore\")|  |   \\n[.pre-commit-config.yaml](/datalab-to/marker/blob/master/.pre-commit-config.yaml \".pre-commit-config.yaml\")| [.pre-commit-config.yaml](/datalab-to/marker/blob/master/.pre-commit-config.yaml \".pre-commit-config.yaml\")|  |   \\n[CLA.md](/datalab-to/marker/blob/master/CLA.md \"CLA.md\")| [CLA.md](/datalab-to/marker/blob/master/CLA.md \"CLA.md\")|  |   \\n[LICENSE](/datalab-to/marker/blob/master/LICENSE \"LICENSE\")| [LICENSE](/datalab-to/marker/blob/master/LICENSE \"LICENSE\")|  |   \\n[MODEL_LICENSE](/datalab-to/marker/blob/master/MODEL_LICENSE \"MODEL_LICENSE\")| [MODEL_LICENSE](/datalab-to/marker/blob/master/MODEL_LICENSE \"MODEL_LICENSE\")|  |   \\n[README.md](/datalab-to/marker/blob/master/README.md \"README.md\")| [README.md](/datalab-to/marker/blob/master/README.md \"README.md\")|  |   \\n[chunk_convert.py](/datalab-to/marker/blob/master/chunk_convert.py \"chunk_convert.py\")| [chunk_convert.py](/datalab-to/marker/blob/master/chunk_convert.py \"chunk_convert.py\")|  |   \\n[convert.py](/datalab-to/marker/blob/master/convert.py \"convert.py\")| [convert.py](/datalab-to/marker/blob/master/convert.py \"convert.py\")|  |   \\n[convert_single.py](/datalab-to/marker/blob/master/convert_single.py \"convert_single.py\")| [convert_single.py](/datalab-to/marker/blob/master/convert_single.py \"convert_single.py\")|  |   \\n[extraction_app.py](/datalab-to/marker/blob/master/extraction_app.py \"extraction_app.py\")| [extraction_app.py](/datalab-to/marker/blob/master/extraction_app.py \"extraction_app.py\")|  |   \\n[marker_app.py](/datalab-to/marker/blob/master/marker_app.py \"marker_app.py\")| [marker_app.py](/datalab-to/marker/blob/master/marker_app.py \"marker_app.py\")|  |   \\n[marker_server.py](/datalab-to/marker/blob/master/marker_server.py \"marker_server.py\")| [marker_server.py](/datalab-to/marker/blob/master/marker_server.py \"marker_server.py\")|  |   \\n[poetry.lock](/datalab-to/marker/blob/master/poetry.lock \"poetry.lock\")| [poetry.lock](/datalab-to/marker/blob/master/poetry.lock \"poetry.lock\")|  |   \\n[pyproject.toml](/datalab-to/marker/blob/master/pyproject.toml \"pyproject.toml\")| [pyproject.toml](/datalab-to/marker/blob/master/pyproject.toml \"pyproject.toml\")|  |   \\n[pytest.ini](/datalab-to/marker/blob/master/pytest.ini \"pytest.ini\")| [pytest.ini](/datalab-to/marker/blob/master/pytest.ini \"pytest.ini\")|  |   \\nView all files  \\n  \\n## Repository files navigation\\n\\n  * README\\n  * GPL-3.0 license\\n\\n# Marker\\n\\nMarker converts documents to markdown, JSON, chunks, and HTML quickly and accurately.\\n\\n  * Converts PDF, image, PPTX, DOCX, XLSX, HTML, EPUB files in all languages\\n  * Formats tables, forms, equations, inline math, links, references, and code blocks\\n  * Extracts and saves images\\n  * Removes headers/footers/other artifacts\\n  * Extensible with your own formatting and logic\\n  * Does structured extraction, given a JSON schema (beta)\\n  * Optionally boost accuracy with LLMs (and your own prompt)\\n  * Works on GPU, CPU, or MPS\\n\\nFor our managed API or on-prem document intelligence solution, check out [our platform here](https://datalab.to?utm_source=gh-marker).\\n\\n## Performance\\n\\n[![](/datalab-to/marker/raw/master/data/images/overall.png)](/datalab-to/marker/blob/master/data/images/overall.png)\\n\\nMarker benchmarks favorably compared to cloud services like Llamaparse and Mathpix, as well as other open source tools.\\n\\nThe above results are running single PDF pages serially. Marker is significantly faster when running in batch mode, with a projected throughput of 25 pages/second on an H100.\\n\\nSee below for detailed speed and accuracy benchmarks, and instructions on how to run your own benchmarks.\\n\\n## Hybrid Mode\\n\\nFor the highest accuracy, pass the `--use_llm` flag to use an LLM alongside marker. This will do things like merge tables across pages, handle inline math, format tables properly, and extract values from forms. It can use any gemini or ollama model. By default, it uses `gemini-2.0-flash`. See below for details.\\n\\nHere is a table benchmark comparing marker, gemini flash alone, and marker with use_llm:\\n\\n[![](/datalab-to/marker/raw/master/data/images/table.png)](/datalab-to/marker/blob/master/data/images/table.png)\\n\\nAs you can see, the use_llm mode offers higher accuracy than marker or gemini alone.\\n\\n## Examples\\n\\nPDF | File type | Markdown | JSON  \\n---|---|---|---  \\n[Think Python](https://greenteapress.com/thinkpython/thinkpython.pdf) | Textbook | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/markdown/thinkpython/thinkpython.md) | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/json/thinkpython.json)  \\n[Switch Transformers](https://arxiv.org/pdf/2101.03961.pdf) | arXiv paper | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/markdown/switch_transformers/switch_trans.md) | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/json/switch_trans.json)  \\n[Multi-column CNN](https://arxiv.org/pdf/1804.07821.pdf) | arXiv paper | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/markdown/multicolcnn/multicolcnn.md) | [View](https://github.com/VikParuchuri/marker/blob/master/data/examples/json/multicolcnn.json)  \\n  \\n# Commercial usage\\n\\nOur model weights use a modified AI Pubs Open Rail-M license (free for research, personal use, and startups under $2M funding/revenue) and our code is GPL. For broader commercial licensing or to remove GPL requirements, visit our pricing page [here](https://www.datalab.to/pricing?utm_source=gh-marker).\\n\\n# Hosted API & On-prem\\n\\nThere\\'s a [hosted API](https://www.datalab.to?utm_source=gh-marker) and [painless on-prem solution](https://www.datalab.to/blog/self-serve-on-prem-licensing) for marker - it\\'s free to sign up, and we\\'ll throw in credits for you to test it out.\\n\\nThe API:\\n\\n  * Supports PDF, image, PPT, PPTX, DOC, DOCX, XLS, XLSX, HTML, EPUB files\\n  * Is 1/4th the price of leading cloud-based competitors\\n  * Fast - ~15s for a 250 page PDF\\n  * Supports LLM mode\\n  * High uptime (99.99%)\\n\\n# Community\\n\\n[Discord](https://discord.gg//KuZwXNGnfH) is where we discuss future development.\\n\\n# Installation\\n\\nYou\\'ll need python 3.10+ and [PyTorch](https://pytorch.org/get-started/locally/).\\n\\nInstall with:\\n\\n```\\npip install marker-pdf\\n```\\n\\nIf you want to use marker on documents other than PDFs, you will need to install additional dependencies with:\\n\\n```\\npip install marker-pdf[full]\\n```\\n\\n# Usage\\n\\nFirst, some configuration:\\n\\n  * Your torch device will be automatically detected, but you can override this. For example, `TORCH_DEVICE=cuda`.\\n  * Some PDFs, even digital ones, have bad text in them. Set `--force_ocr` to force OCR on all lines, or the `strip_existing_ocr` to keep all digital text, and strip out any existing OCR text.\\n  * If you care about inline math, set `force_ocr` to convert inline math to LaTeX.\\n\\n## Interactive App\\n\\nI\\'ve included a streamlit app that lets you interactively try marker with some basic options. Run it with:\\n\\n```\\npip install streamlit streamlit-ace\\nmarker_gui\\n```\\n\\n## Convert a single file\\n\\n```\\nmarker_single /path/to/file.pdf\\n```\\n\\nYou can pass in PDFs or images.\\n\\nOptions:\\n\\n  * `--page_range TEXT`: Specify which pages to process. Accepts comma-separated page numbers and ranges. Example: `--page_range \"0,5-10,20\"` will process pages 0, 5 through 10, and page 20.\\n  * `--output_format [markdown|json|html|chunks]`: Specify the format for the output results.\\n  * `--output_dir PATH`: Directory where output files will be saved. Defaults to the value specified in settings.OUTPUT_DIR.\\n  * `--paginate_output`: Paginates the output, using `\\\\n\\\\n{PAGE_NUMBER}` followed by `-` * 48, then `\\\\n\\\\n`\\n  * `--use_llm`: Uses an LLM to improve accuracy. You will need to configure the LLM backend - see below.\\n  * `--force_ocr`: Force OCR processing on the entire document, even for pages that might contain extractable text. This will also format inline math properly.\\n  * `--block_correction_prompt`: if LLM mode is active, an optional prompt that will be used to correct the output of marker. This is useful for custom formatting or logic that you want to apply to the output.\\n  * `--strip_existing_ocr`: Remove all existing OCR text in the document and re-OCR with surya.\\n  * `--redo_inline_math`: If you want the absolute highest quality inline math conversion, use this along with `--use_llm`.\\n  * `--disable_image_extraction`: Don\\'t extract images from the PDF. If you also specify `--use_llm`, then images will be replaced with a description.\\n  * `--debug`: Enable debug mode for additional logging and diagnostic information.\\n  * `--processors TEXT`: Override the default processors by providing their full module paths, separated by commas. Example: `--processors \"module1.processor1,module2.processor2\"`\\n  * `--config_json PATH`: Path to a JSON configuration file containing additional settings.\\n  * `config --help`: List all available builders, processors, and converters, and their associated configuration. These values can be used to build a JSON configuration file for additional tweaking of marker defaults.\\n  * `--converter_cls`: One of `marker.converters.pdf.PdfConverter` (default) or `marker.converters.table.TableConverter`. The `PdfConverter` will convert the whole PDF, the `TableConverter` will only extract and convert tables.\\n  * `--llm_service`: Which llm service to use if `--use_llm` is passed. This defaults to `marker.services.gemini.GoogleGeminiService`.\\n  * `--help`: see all of the flags that can be passed into marker. (it supports many more options then are listed above)\\n\\nThe list of supported languages for surya OCR is [here](https://github.com/VikParuchuri/surya/blob/master/surya/recognition/languages.py). If you don\\'t need OCR, marker can work with any language.\\n\\n## Convert multiple files\\n\\n```\\nmarker /path/to/input/folder\\n```\\n\\n  * `marker` supports all the same options from `marker_single` above.\\n  * `--workers` is the number of conversion workers to run simultaneously. This is automatically set by default, but you can increase it to increase throughput, at the cost of more CPU/GPU usage. Marker will use 5GB of VRAM per worker at the peak, and 3.5GB average.\\n\\n## Convert multiple files on multiple GPUs\\n\\n```\\nNUM_DEVICES=4 NUM_WORKERS=15 marker_chunk_convert ../pdf_in ../md_out\\n```\\n\\n  * `NUM_DEVICES` is the number of GPUs to use. Should be `2` or greater.\\n  * `NUM_WORKERS` is the number of parallel processes to run on each GPU.\\n\\n## Use from python\\n\\nSee the `PdfConverter` class at `marker/converters/pdf.py` function for additional arguments that can be passed.\\n\\n```\\nfrom marker.converters.pdf import PdfConverter\\nfrom marker.models import create_model_dict\\nfrom marker.output import text_from_rendered\\n\\nconverter = PdfConverter(\\n    artifact_dict=create_model_dict(),\\n)\\nrendered = converter(\"FILEPATH\")\\ntext, _, images = text_from_rendered(rendered)\\n```\\n\\n`rendered` will be a pydantic basemodel with different properties depending on the output type requested. With markdown output (default), you\\'ll have the properties `markdown`, `metadata`, and `images`. For json output, you\\'ll have `children`, `block_type`, and `metadata`.\\n\\n### Custom configuration\\n\\nYou can pass configuration using the `ConfigParser`. To see all availa<TRUNCATED>"
}
```

</details>

Perfect! I've read the Marker repo. Key findings for your use case:

**Marker has built-in chunking support!** There's a `chunk_convert.py` file and you can output in "chunks" format using `--output_format chunks`.

The chunks format:
- Flattens everything into a single list (not a tree)
- Only top-level blocks from each page
- Includes full HTML of each block
- Designed specifically for RAG applications

**For tables specifically:**
- Marker extracts and formats tables automatically
- Can use `--use_llm` flag to improve table accuracy significantly (0.816 → 0.907 score)
- Has a dedicated `TableConverter` for table-only extraction

Would you like to try using Marker's chunking functionality to replace your current document processing?

I want to first try, lets try it on the strubell2019 document

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! Let's start by installing marker and testing it on that document.

First, install marker:

```python
!pip install marker-pdf
```

Then try converting the strubell2019.pdf with chunking output:

```python
!marker_single wattbot_data/strubell2019.pdf --output_format chunks --output_dir marker_output
```

This will create chunks and save them to the `marker_output` directory. Want to give it a try and see what the output looks like?

In [ ]:
!pip install marker-pdf

  Using cached openai-1.109.1-py3-none-any.whl.metadata (29 kB)


Using cached openai-1.109.1-py3-none-any.whl (948 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.8 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/4.5 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 4.5/4.5 MB 280.7 MB/s eta 0:00:01

...

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━ 18/20 [surya-ocr]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━ 19/20 [marker-pdf]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20/20 [marker-pdf]


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
reward-kit 0.4.1 requires openai==1.78.1, but you have openai 1.109.1 which is incompatible.
claudette 0.3.10 requires anthropic>=0.52, but you have anthropic 0.46.0 which is incompatible.


Is it possible to use python equivalent instead of cli command of marker_single

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! From the README I read earlier, you can use Marker directly in Python. Here's the basic approach:

```python
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter("wattbot_data/strubell2019.pdf")
```

For chunks specifically, you'd need to specify the output format. Would you like to try this and see what properties the `rendered` object has?

In [ ]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter("wattbot_data/strubell2019.pdf")

In [ ]:
len(rendered)

lets try fitz, i want to know if we can give chunk size and overlap

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

PyMuPDF (fitz) is primarily a PDF rendering library - it doesn't have built-in chunking with overlap like some text processing libraries. However, you can:

1. Extract text from the PDF with fitz
2. Manually implement chunking with overlap on the extracted text

Would you like to try extracting text with fitz first to see what you get, or would you prefer to continue exploring Marker's chunking capabilities since it's already designed for this use case?

lets try with fitz because marker is difficult to run, takes lot of resource. we can try marker if fitz does not workout

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good call - fitz is much lighter weight. Let's start by extracting text from the strubell2019 PDF with fitz:

```python
import fitz
doc = fitz.open(data_path/'strubell2019.pdf')
```

Then you can extract text page by page or all at once. Would you like to try extracting from one page first to see what the output looks like?

In [ ]:
import fitz
doc = fitz.open(data_path/'strubell2019.pdf')


sure

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Try extracting text from the first page:

```python
page = doc[0]
text = page.get_text()
print(text)
```

This will show you what fitz extracts. Take a look at the output and see how it compares to what you got with `rd.read_pdf`.

In [ ]:
page = doc[0]
text = page.get_text()

In [ ]:
print(text)

arXiv:1906.02243v1  [cs.CL]  5 Jun 2019
Energy and Policy Considerations for Deep Learning in NLP
Emma Strubell
Ananya Ganesh
Andrew McCallum
College of Information and Computer Sciences
University of Massachusetts Amherst
{strubell, aganesh, mccallum}@cs.umass.edu
Abstract
Recent progress in hardware and methodol-
ogy for training neural networks has ushered
in a new generation of large networks trained
on abundant data.
These models have ob-
tained notable gains in accuracy across many
NLP tasks. However, these accuracy improve-
ments depend on the availability of exception-
ally large computational resources that neces-
sitate similarly substantial energy consump-
tion. As a result these models are costly to
train and develop, both ﬁnancially, due to the
cost of hardware and electricity or cloud com-
pute time, and environmentally, due to the car-
bon footprint required to fuel modern tensor
processing hardware. In this paper we bring
this issue to the attention of NLP researchers
b

The table information is extracted but how to know rows and column

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good observation! Fitz extracts the table as plain text without preserving the row/column structure. To get structured table data, you'd need to:

1. **Use fitz's table detection**: `page.find_tables()` - this can identify tables and extract them as structured data
2. **Parse the text manually** - look for patterns in spacing/alignment

Would you like to try `page.find_tables()` to see if it can detect the table structure on this page?

In [ ]:
tables = page.find_tables(); tables

<pymupdf.table.TableFinder>

what is next?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

The `tables` object is a TableFinder. To see the actual tables, try:

```python
tables.tables
```

This will show you a list of detected tables. If there are any, you can then extract them as structured data.

In [ ]:
tables.tables

[]

it is empty

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Fitz's table detection didn't find the table on that page - probably because it's formatted as plain text without borders or clear structure.

You have a few options:

1. **Try adjusting fitz's table detection settings** - it has parameters you can tune
2. **Extract tables as images and use OCR** - fitz can extract page regions as images
3. **Go back to Marker** - it uses ML models specifically trained for table detection
4. **Use a dedicated table extraction library** like camelot or tabula

Since fitz isn't detecting the tables automatically, would you like to try one of these alternatives, or would you prefer to work with the text as-is and implement your own chunking strategy?

using marker will return markdown. Do you know how to chunk a markdow?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Yes! Markdown can be chunked in several ways:

1. **By headers** - Split on `#`, `##`, `###` etc. to keep related sections together
2. **By paragraphs** - Split on `\n\n` (double newlines)
3. **By semantic units** - Use a library like `langchain` or `llama-index` that has markdown-aware chunkers
4. **Fixed size with overlap** - Just like regular text, but preserve markdown formatting

For tables specifically, Marker converts them to markdown table format, which you could:
- Keep as complete units (don't split tables across chunks)
- Parse into structured data

Would you like to try getting Marker's markdown output first and then explore how to chunk it effectively?

I have got access to datalab now. I want to use their api for the markdown extraction

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great! Datalab is the hosted API for Marker. Would you like me to look up the Datalab API documentation to see how to use it for markdown extraction?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! I found the Datalab API documentation. Here's the key info:

[*](https://documentation.datalab.to/docs/welcome/api "API Overview - Datalab Documentation") The marker endpoint is at `/api/v1/marker`, and you can submit requests with parameters like `output_format` (set to 'markdown'), `force_ocr`, `use_llm`, etc.

Basic Python example:
```python
import requests

url = "https://www.datalab.to/api/v1/marker"
form_data = {
    'file': ('test.pdf', open('path/to/file.pdf', 'rb'), 'application/pdf'),
    'output_format': (None, 'markdown'),  # or 'chunks'
}
headers = {"X-Api-Key": "YOUR_API_KEY"}
response = requests.post(url, files=form_data, headers=headers)
```

[*](https://documentation.datalab.to/docs/welcome/api "API Overview - Datalab Documentation") This returns a `request_id` that you use to retrieve the result later.

Would you like to try calling the API with one of your PDFs?

I have the following code
#| export
class HybridSearch:
    def __init__(self, all_chunks, dense_embds, tokenized_corpus):
        fc.store_attr()
        self.bm25 = BM25Okapi(self.tokenized_corpus)

    def search(self, query, limit=5):
        query_emb = embedder.create_dembs(texts=[query])[0]
        tokenized_query = query.lower().split()
        results = self.bm25.get_top_n(tokenized_query, self.all_chunks, n=limit)
        scores = cosine_similarity([query_emb], self.dense_embds)[0]
        # print(len(scores), scores[0], len(scores[0]))
        idx = np.argsort(scores)[-limit:][::-1]
        results.extend(self.all_chunks[idx])
        return results

I want to use qwen reranker, also I want to include the preceding and next chunk for each chunk in the results

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good improvements! Let me break this down into steps:

**For reranking with Qwen:**
You'll need to combine the BM25 and dense results first, then pass them to the reranker to get the final top-k.

**For including preceding/next chunks:**
You'll need to track chunk indices so you can grab adjacent chunks.

Which would you like to tackle first - adding the reranker or implementing the context window (prev/next chunks)?

Including the prev and next chunks 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good choice! To include prev/next chunks, you'll need to:

1. Track chunk indices in your results
2. Fetch adjacent chunks based on those indices
3. Handle edge cases (first/last chunks in a document)

Here's a helper method you could add to your `HybridSearch` class:

```python
def get_context_chunks(self, chunk_indices, window=1):
    """Get chunks with surrounding context"""
    result_chunks = []
    seen = set()
    
    for idx in chunk_indices:
        start = max(0, idx - window)
        end = min(len(self.all_chunks), idx + window + 1)
        
        for i in range(start, end):
            if i not in seen:
                result_chunks.append((i, self.all_chunks[i]))
                seen.add(i)
    
    return [chunk for _, chunk in sorted(result_chunks)]
```

However, there's one issue - you probably only want to include adjacent chunks from the **same document**. Should we add a check for `doc_id` to prevent mixing chunks from different documents?

Yes

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Here's an updated version that respects document boundaries:

```python
def get_context_chunks(self, chunk_indices, window=1):
    """Get chunks with surrounding context, respecting document boundaries"""
    result_chunks = []
    seen = set()
    
    for idx in chunk_indices:
        current_doc_id = self.all_chunks[idx].doc_id
        
        # Find adjacent chunks in same document
        for offset in range(-window, window + 1):
            new_idx = idx + offset
            
            # Check bounds and same document
            if (0 <= new_idx < len(self.all_chunks) and 
                new_idx not in seen and
                self.all_chunks[new_idx].doc_id == current_doc_id):
                
                result_chunks.append((new_idx, self.all_chunks[new_idx]))
                seen.add(new_idx)
    
    return [chunk for _, chunk in sorted(result_chunks)]
```

Then update your `search` method to use it:

```python
def search(self, query, limit=5, window=1):
    # ... your existing search logic ...
    
    # Combine and get top indices
    combined_indices = list(set(bm25_indices + dense_indices))[:limit]
    
    # Get chunks with context
    return self.get_context_chunks(combined_indices, window=window)
```

Would you like to integrate this into your current code?

Please follow fastai coding style

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's the fastai-style version:

```python
def get_context_chunks(self, idxs, window=1):
    """Get chunks with surrounding context, respecting doc boundaries"""
    seen, res = set(), []
    for i in idxs:
        doc_id = self.all_chunks[i].doc_id
        for j in range(max(0, i-window), min(len(self.all_chunks), i+window+1)):
            if j not in seen and self.all_chunks[j].doc_id == doc_id:
                res.append((j, self.all_chunks[j]))
                seen.add(j)
    return fc.L(sorted(res)).itemgot(1)
```

And the updated `search` method:

```python
def search(self, query, limit=5, window=1):
    q_emb = embedder.create_dembs(texts=[query])[0]
    tok_q = query.lower().split()
    
    bm25_res = self.bm25.get_top_n(tok_q, range(len(self.all_chunks)), n=limit)
    dense_scores = cosine_similarity([q_emb], self.dense_embds)[0]
    dense_idxs = np.argsort(dense_scores)[-limit:][::-1].tolist()
    
    idxs = list(set(bm25_res + dense_idxs))[:limit]
    return self.get_context_chunks(idxs, window=window)
```

Note: I changed the BM25 call to return indices instead of chunks directly so we can use them for context lookup. Would you like to try this?

Can you please make the get_context_chunks simpler with fixed window of 1?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Sure! Here's a simpler version with `window=1` fixed:

```python
def get_context_chunks(self, idxs):
    """Get chunks with prev/next context, respecting doc boundaries"""
    seen, res = set(), []
    for i in idxs:
        doc_id = self.all_chunks[i].doc_id
        for j in [i-1, i, i+1]:
            if 0 <= j < len(self.all_chunks) and j not in seen and self.all_chunks[j].doc_id == doc_id:
                res.append((j, self.all_chunks[j]))
                seen.add(j)
    return fc.L(sorted(res)).itemgot(1)
```

Much cleaner! This checks only the immediate prev/current/next chunks for each result.

now lets use the qwen reranker

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great! For Qwen reranker, you'll need to:

1. Install the model (if using locally) or use an API
2. Get initial candidates from BM25 + dense search (more than `limit`, e.g., `limit * 2`)
3. Pass candidates to reranker with the query
4. Return top `limit` reranked results

Are you planning to use:
- **Local Qwen reranker** (via transformers/sentence-transformers)?
- **API-based** (like through Fireworks or another provider)?

Which approach would you prefer?

Yes 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠

Found it! [*](https://docs.fireworks.ai/guides/querying-embeddings-models "Embeddings & Reranking - Fireworks AI Docs") The Fireworks reranking API uses the endpoint `https://api.fireworks.ai/inference/v1/rerank` with model `fireworks/qwen3-reranker-8b`.

Here's how to add the reranker to your `HybridSearch` class:

```python
import requests

def rerank(self, query, chunks, limit=5):
    """Rerank chunks using Fireworks Qwen3 reranker"""
    docs = [c.text for c in chunks]
    res = requests.post(
        "https://api.fireworks.ai/inference/v1/rerank",
        json={
            "model": "fireworks/qwen3-reranker-8b",
            "query": query,
            "documents": docs,
            "top_n": limit,
            "return_documents": True
        },
        headers={
            "Authorization": f"Bearer {os.environ['FIREWORKS_API_KEY']}",
            "Content-Type": "application/json"
        }
    )
    ranked_idxs = [r['index'] for r in res.json()['results']]
    return fc.L([chunks[i] for i in ranked_idxs])
```

Then update your `search` method to use it:

```python
def search(self, query, limit=5):
    # Get more candidates than needed
    candidates = self._get_candidates(query, limit=limit*3)
    # Rerank and return top results
    return self.rerank(query, candidates, limit=limit)
```

Would you like to integrate this into your class?

You can also use the openai spec instead of the request library

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good point! Here's the simpler version using the OpenAI client:

```python
def rerank(self, query, chunks, limit=5):
    """Rerank chunks using Fireworks Qwen3 reranker"""
    docs = [c.text for c in chunks]
    res = openai_cli.post(
        "/rerank",
        body={
            "model": "fireworks/qwen3-reranker-8b",
            "query": query,
            "documents": docs,
            "top_n": limit
        },
        cast_to=dict
    )
    ranked_idxs = [r['index'] for r in res['results']]
    return fc.L([chunks[i] for i in ranked_idxs])
```

Note: If the `.post()` method doesn't work directly, you might need to use:

```python
import httpx

def rerank(self, query, chunks, limit=5):
    docs = [c.text for c in chunks]
    res = httpx.post(
        f"{openai_cli.base_url}rerank",
        json={"model": "fireworks/qwen3-reranker-8b", "query": query, "documents": docs, "top_n": limit},
        headers={"Authorization": f"Bearer {os.environ['FIREWORKS_API_KEY']}"}
    ).json()
    return fc.L([chunks[r['index']] for r in res['results']])
```

Would you like to try one of these approaches?